# Transformation Codes

# 1. binomial transform

In [1]:
import pandas as pd

# Dosya yolları
file_binomial = "1. binomial_transformed_full.csv"
file_stripped = "stripped.csv"

# Dosyaları uygun ayraçla yükle (örnek olarak virgül ve noktalı virgül denenecek)
try:
    df_binomial = pd.read_csv(file_binomial, sep=';', header=None)
except:
    df_binomial = pd.read_csv(file_binomial, sep=',', header=None)

try:
    df_stripped = pd.read_csv(file_stripped, sep=';', header=None)
except:
    df_stripped = pd.read_csv(file_stripped, sep=',', header=None)

# İlk 10 elemanı karşılaştırmak için dizileri liste olarak al
binomial_sequences = df_binomial.values.tolist()
stripped_sequences = df_stripped.values.tolist()

# İlk 10 elemanı aynı olan bir dizi var mı kontrol eden fonksiyon
def has_common_prefix(seq, reference_sequences, prefix_length=10):
    prefix = seq[:prefix_length]
    for ref_seq in reference_sequences:
        if prefix == ref_seq[:prefix_length]:
            return True
    return False

# Süzülmüş dizileri tutacağımız liste
filtered_sequences = []

# Her binomial dizisini stripped dizileri ile karşılaştır
for seq in binomial_sequences:
    if not has_common_prefix(seq, stripped_sequences):
        filtered_sequences.append(seq)

# Süzülen dizileri yeni bir CSV dosyasına kaydet
filtered_df = pd.DataFrame(filtered_sequences)
filtered_df.to_csv("süzme.csv", index=False, header=False)

print("✅ Süzülen diziler 'süzme.csv' dosyasına kaydedildi.")


✅ Süzülen diziler 'süzme.csv' dosyasına kaydedildi.


In [ ]:
import pandas as pd
from math import comb

# Binomial transform function
def binomial_transform(seq):
    n = len(seq)
    transformed = []
    for i in range(n):
        val = sum(comb(i, j) * seq[j] for j in range(i + 1))
        transformed.append(val)
    return transformed

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply binomial transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 20:
        continue
    transformed = binomial_transform(original[:20])
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "BinomialFirst20": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "binomial_transformed_only.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path

# 2.	trv_i_bin (ters binom dönüşümü)  inverse binomial transform

In [3]:
import pandas as pd
from math import comb

# Inverse binomial transform function
def inverse_binomial_transform(seq):
    seq = seq[:]  # Make a copy to avoid modifying the original list
    n = len(seq)
    for i in range(1, n):
        for j in range(i):
            seq[i] -= comb(i, j) * seq[j]
    return seq

# Read original stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue  # Skip lines with invalid integers

# Apply inverse binomial transform
inverted_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    inverted = inverse_binomial_transform(original)
    inverted_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseBinomial": inverted
    })

# Save result to CSV
inverted_df = pd.DataFrame(inverted_sequences)
inverted_file_path = "inverse_binomial_transformed_full.csv"
inverted_df.to_csv(inverted_file_path, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {inverted_file_path}")


Dönüştürülmüş dosya kaydedildi: inverse_binomial_transformed_full.csv


# 3.  characteristic function of A

In [4]:
import pandas as pd

# Python karşılığı: characteristic function with explicit output length n
def trv_char(seq, n):
    B = [0] * n
    for v in seq:
        if isinstance(v, int) and 0 <= v < n:
            B[v] = 1
    return B

# stripped.csv dosyasını oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Uygulanacak maksimum karakteristik vektör uzunluğu
MAX_OUTPUT_LENGTH = 1000  # bu, örnek uzunluk, istersen artırabilirsin

# Her diziye trv_char fonksiyonu uygula
char_results = []
for entry in sequences:
    seq = entry["Sequence"]
    char_seq = trv_char(seq, MAX_OUTPUT_LENGTH)
    char_results.append({
        "SequenceID": entry["SequenceID"],
        "Characteristic": char_seq
    })

# DataFrame'e çevir ve kaydet
char_df = pd.DataFrame(char_results)
output_file = "characteristic_function_fixed_length.csv"
char_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✓ Karakteristik dönüşüm uygulandı ve kaydedildi: {output_file}")


✓ Karakteristik dönüşüm uygulandı ve kaydedildi: characteristic_function_fixed_length.csv


# 4. /* inverse characteristic function of A

In [5]:
import pandas as pd

# Inverse characteristic function: return list of indices where value != 0
def inverse_characteristic_function(seq):
    return [i for i, val in enumerate(seq) if val != 0]

# stripped.csv dosyasını oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüşüm uygula
inverse_char_results = []
for entry in sequences:
    char_vec = entry["Sequence"]  # bu durumda 'Sequence' characteristic vector olarak varsayılıyor
    support_list = inverse_characteristic_function(char_vec)
    inverse_char_results.append({
        "SequenceID": entry["SequenceID"],
        "InverseCharacteristic": support_list
    })

# Sonucu kaydet
output_df = pd.DataFrame(inverse_char_results)
output_file = "inverse_characteristic_output.csv"
output_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✓ Inverse characteristic dönüşümü kaydedildi: {output_file}")


✓ Inverse characteristic dönüşümü kaydedildi: inverse_characteristic_output.csv


# 5. /* continued fraction numerator transform

In [6]:
import pandas as pd

# Continued Fraction Numerator Transform
def continued_fraction_numerator(seq):
    seq = seq[:]  # Copy to avoid modifying original
    n = len(seq)
    if n > 1:
        seq[1] = seq[1] * seq[0] + 1
    for i in range(2, n):
        seq[i] = seq[i] * seq[i - 1] + seq[i - 2]
    return seq

# Read stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue  # skip invalid lines

# Apply transformation
cf_num_results = []
for entry in sequences:
    seq = entry["Sequence"]
    transformed = continued_fraction_numerator(seq)
    cf_num_results.append({
        "SequenceID": entry["SequenceID"],
        "ContinuedFractionNumerators": transformed
    })

# Save to CSV
cf_df = pd.DataFrame(cf_num_results)
output_file = "continued_fraction_numerators.csv"
cf_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✓ Continued fraction numerator transform saved to: {output_file}")


✓ Continued fraction numerator transform saved to: continued_fraction_numerators.csv


# 6. continued fraction denominator transform

In [7]:
import pandas as pd

# Continued Fraction Denominator Transform
def continued_fraction_denominator(seq):
    seq = seq[:]  # copy to avoid modifying original
    n = len(seq)
    if n >= 1:
        seq[0] = 1
    for i in range(2, n):
        seq[i] = seq[i] * seq[i - 1] + seq[i - 2]
    return seq

# stripped.csv dosyasını oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue  # skip invalid lines

# Dönüşümü uygula
denominator_results = []
for entry in sequences:
    seq = entry["Sequence"]
    transformed = continued_fraction_denominator(seq)
    denominator_results.append({
        "SequenceID": entry["SequenceID"],
        "ContinuedFractionDenominators": transformed
    })

# CSV olarak kaydet
output_df = pd.DataFrame(denominator_results)
output_file = "continued_fraction_denominators.csv"
output_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✓ Continued fraction denominator transform saved to: {output_file}")


✓ Continued fraction denominator transform saved to: continued_fraction_denominators.csv


# 7. cycle transform
Bu yalnış Çıkıyor sonra yine bak??????

# 8. inverse cycle transform
Bu yalnış Çıkıyor sonra yine bak??????

# 9. difference transform

In [13]:
import pandas as pd

# Difference transform function
def difference_transform(seq):
    if not seq:
        return []
    result = [seq[0]]
    for i in range(1, len(seq)):
        result.append(seq[i] - seq[i - 1])
    return result

# Read original stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue  # Skip lines with invalid integers

# Apply difference transform
transformed_sequences = []
for entry in sequences:
    transformed = difference_transform(entry["Sequence"])
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "DifferenceTransform": transformed
    })

# Save result to CSV
df = pd.DataFrame(transformed_sequences)
output_path = "difference_transformed_full.csv"
df.to_csv(output_path, index=False)
print(f"Dönüştürülmüş dosya kaydedildi: {output_path}")


Dönüştürülmüş dosya kaydedildi: difference_transformed_full.csv


# 10. euler transform
Bu yalnış Çıkıyor sonra yine bak??????

# 11. inverse euler transform
Bu yalnış Çıkıyor sonra yine bak??????

# 12.  euler_2 transform

In [21]:
import pandas as pd
import numpy as np

# Polynomial multiplication helper (truncates to degree n)
def poly_mult(p1, p2, n):
    result = [0] * n
    for i in range(len(p1)):
        for j in range(len(p2)):
            if i + j < n:
                result[i + j] += p1[i] * p2[j]
    return result

# Euler_2 transform function
def euler_2_transform(seq):
    n = len(seq)
    A = seq[:]

    # B = generating function
    B = A

    # B^2
    B2 = poly_mult(B, B, n)

    # B(x^2)
    B_x2 = [0] * n
    for i in range(len(B)):
        if 2 * i < n:
            B_x2[2 * i] = B[i]

    # (B^2 + B(x^2)) / 2
    result = [(B2[i] + B_x2[i]) // 2 for i in range(n)]
    return result

# Read original stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue  # Skip lines with invalid integers

# Apply euler_2 transform
euler2_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = euler_2_transform(original)
    euler2_sequences.append({
        "SequenceID": entry["SequenceID"],
        "Euler2": transformed
    })

# Save result to CSV
df = pd.DataFrame(euler2_sequences)
output_path = "euler2_transformed_full.csv"
df.to_csv(output_path, index=False)
print(f"Dönüştürülmüş dosya kaydedildi: {output_path}")


Dönüştürülmüş dosya kaydedildi: euler2_transformed_full.csv


# 13. euler_2 transform for matrices

In [22]:
import pandas as pd

# Euler_2 transform (symmetric square)
def euler_2_transform(seq):
    n = len(seq)
    result = [0] * n
    for k in range(n):
        s = 0
        for i in range(k + 1):
            j = k - i
            if i < len(seq) and j < len(seq):
                s += seq[i] * seq[j]
        if k % 2 == 0 and (k // 2) < len(seq):
            s += seq[k // 2]
        result[k] = s // 2
    return result

# CSV dosyasını oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüşümü uygula
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = euler_2_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        **{f"a{i}": val for i, val in enumerate(transformed)}
    })

# CSV olarak kaydet
df = pd.DataFrame(transformed_sequences)
output_path = "euler2_transformed.csv"
df.to_csv(output_path, index=False)
print(f"Euler_2 dönüşümü tamamlandı: {output_path}")


Euler_2 dönüşümü tamamlandı: euler2_transformed.csv


# 14. euler product transform
Bu yalnış Çıkıyor sonra yine bak??????

# 15. inverse euler product transform

# 16. exponential transform

Uzun dizilerin sadece ilk birkaç terimiyle işlem yapmak belleği korur. Bellek taşmasının önüne geçmek için uzun dizileri kısıtladık. 

Hatanız bir MemoryError, yani belleğiniz (RAM) aşırı kullanılmış ve işlem sırasında SymPy'nin series(exp(...)) genişlemesi bellekte yeterli alan bulamamış. Bu büyük olasılıkla şu nedenlerle oluyor:


In [ ]:
import pandas as pd
from math import comb

# Combinatorial exponential transform
def exponential_transform(seq):
    n = len(seq)
    b = [1]  # b_0 = 1
    for i in range(1, n):
        val = 0
        for k in range(1, i + 1):
            val += seq[k] * b[i - k] * comb(i - 1, k - 1)
        b.append(val)
    return b

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply exponential transform
MAX_TERMS = 20  # Optional limit to avoid memory issues
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    truncated = original[:MAX_TERMS]
    transformed = exponential_transform(truncated)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "ExponentialTransform": transformed
    })

# Save result to CSV
transformed_df = pd.DataFrame(transformed_sequences)
output_file = "exponential_transformed.csv"
transformed_df.to_csv(output_file, index=False)
print(f"Dönüştürülmüş dosya kaydedildi: {output_file}")


# 17. invert transform

In [ ]:
import pandas as pd
from sympy import Symbol, Poly, series, Rational

# Invert transform function (trv_invert)
def invert_transform(seq):
    x = Symbol('x')
    n = len(seq)

    # A[1] assumed to be 0 (i.e., seq[0] = 0), result will have b_0 = 1
    if seq[0] != 0:
        seq = [0] + seq  # Ensure a₀ = 0

    # Create generating function (Ser(A,x) - A[1])
    gen_func = sum(Rational(seq[i], 1) * x**i for i in range(len(seq)))
    B = 1 / (1 - gen_func)

    # Get coefficients of 1 / (1 - f(x))
    B_series = series(B, x, 0, n).removeO()
    B_poly = Poly(B_series, x)

    result = [B_poly.coeff_monomial(x**i) for i in range(n)]
    return [int(r) for r in result]

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue  # Skip invalid lines

# Apply invert transform
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 1:
        continue
    transformed = invert_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InvertTransform": transformed
    })

# Save results to CSV
output_file = "invert_transformed.csv"
df = pd.DataFrame(transformed_sequences)
df.to_csv(output_file, index=False)
print(f"Dönüştürülmüş dosya kaydedildi: {output_file}")


# 18. inverse invert transform

In [ ]:
import pandas as pd
from sympy import Symbol, Poly, series, Rational

# Inverse invert transform function
def inverse_invert_transform(seq):
    x = Symbol('x')
    n = len(seq)

    # Assume a₀ = 1 regardless of input
    seq = seq[:]  # copy
    if len(seq) == 0:
        return [0]
    seq[0] = 1  # Enforce a₀ = 1

    # B = Ser(A,x) - A[1] + 1
    gen_func = sum(Rational(seq[i], 1) * x**i for i in range(n))
    B = 1 - 1 / gen_func

    B_series = series(B, x, 0, n).removeO()
    B_poly = Poly(B_series, x)

    result = [B_poly.coeff_monomial(x**i) for i in range(n)]
    return [int(r) for r in result]

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse invert transform and keep only transformed values
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 20:
        continue
    transformed = inverse_invert_transform(original[:20])
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseInvertFirst20": transformed
    })

# Save results to CSV
transformed_df = pd.DataFrame(transformed_sequences)
output_path = "inverse_invert_transformed_only.csv"
transformed_df.to_csv(output_path, index=False)

output_path


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 19. invert product transform

In [ ]:
import pandas as pd
from sympy import divisors, mobius

# Invert Product Transform Function
def invert_product_transform(seq):
    if len(seq) < 3:
        return [0] * len(seq)
    
    # Ignore a_0 and a_1 (assume both are 0), and extract from a_2 onward
    A = seq[2:]
    n = len(A)
    
    # A[1] = 1 (1-based), yani Python'da A[0]
    A[0] = 1
    A = [-a for a in A]  # Apply unary minus

    result = []
    for n_idx in range(1, n + 1):
        term = sum(mobius(n_idx // d) * A[d - 1] for d in divisors(n_idx) if n_idx % d == 0)
        result.append(term)

    return [0] + result  # b_0 = 0

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply invert product transform
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 3:
        continue
    transformed = invert_product_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InvertProduct": transformed
    })

# Save to CSV
df = pd.DataFrame(transformed_sequences)
df.to_csv("invert_product_transformed.csv", index=False)

print("Dönüştürülmüş dosya kaydedildi: invert_product_transformed.csv")


# 20. inverse invert product transform

# 21. invert labeled transform

# 22. inverse invert labeled transform

In [ ]:
import pandas as pd
from math import factorial
from sympy import Symbol, series, expand, factorial as sympy_fact

# Inverse Invert Labeled Transform
def inverse_invert_labeled_transform(seq):
    if len(seq) < 2:
        return [0] * len(seq)

    x = Symbol('x')
    n = len(seq)

    # Generating function G(x) = sum a[i] * x^i / i!
    G = sum(seq[i] * x**i / sympy_fact(i) for i in range(1, n))
    B_expr = 1 - 1 / (1 + G)
    B_series = series(B_expr, x, n=n).removeO()

    transformed = []
    for i in range(n):
        coeff = B_series.coeff(x, i)
        transformed.append(int(coeff * factorial(i)))

    transformed[0] = 0  # enforce b_0 = 0
    return transformed

# Read stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse invert labeled transform
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = inverse_invert_labeled_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InvInvertLabeled": transformed
    })

# Save to CSV
df = pd.DataFrame(transformed_sequences)
output_path = "inverse_invert_labeled_transformed.csv"
df.to_csv(output_path, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {output_path}")


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 23. least inverse transform

Burada dizilerin ilk 20 elemanı alınarak yapılmıştır. Tüm dizi elemanını dönüştürüldüğünde bilgisayar kasıyor. Sonuçlar doğru bulunmuştur.

In [1]:
import pandas as pd

def least_inverse_transform(seq, n, fill=-1):
    B = [fill] * n
    for i, val in enumerate(seq):
        if isinstance(val, int) and 0 <= val < n and B[val] == fill:
            B[val] = i
    return B

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply least inverse transform
output_length = 20
fill_value = -1
transformed_sequences = []

for entry in sequences:
    original = entry["Sequence"]
    transformed = least_inverse_transform(original, output_length, fill_value)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "LeastInverse": transformed
    })

# Save to CSV
df = pd.DataFrame(transformed_sequences)
output_path = "least_inverse_transformed.csv"
df.to_csv(output_path, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {output_path}")


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Dönüştürülmüş dosya kaydedildi: least_inverse_transformed.csv


# 24. logarithmic transform

In [ ]:
import pandas as pd
import numpy as np
from math import factorial
from sympy import Symbol, series, log, expand, Poly

# Logarithmic transform function
def logarithmic_transform(seq):
    x = Symbol('x')
    n = len(seq)
    
    # Build exponential generating function: B(x) = 1 + sum(seq[i] * x^i / i!) for i = 0 to n-1
    B = 1 + sum(seq[i] * x**i / factorial(i) for i in range(n))
    
    # Take log(B(x)) and get its Maclaurin series
    L = log(B).series(x, 0, n).removeO()
    
    # Extract coefficients and multiply by n! to get the integer sequence
    poly = Poly(expand(L), x)
    coeffs = [int(poly.coeff_monomial(x**i) * factorial(i)) for i in range(n)]
    
    return coeffs

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply logarithmic transform and store transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = logarithmic_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "LogarithmicTransform": transformed
    })

# Save to CSV
df = pd.DataFrame(transformed_sequences)
output_path = "logarithmic_transformed.csv"
df.to_csv(output_path, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {output_path}")


# 25. lyndon transform

In [1]:
import pandas as pd
from sympy import mobius, divisors

# Ters Euler dönüşümü (Inverse Euler Transform)
def inverse_euler_transform(seq):
    n = len(seq)
    result = []
    for k in range(1, n + 1):
        s = sum(mobius(k // d) * seq[d - 1] for d in divisors(k) if k % d == 0)
        result.append(s)
    return result

# Lyndon dönüşümü
def lyndon_transform(seq):
    if not seq:
        return []

    # A[0] sıfır varsayılıyor, dizi negatifleniyor
    negated_seq = [-a for a in seq]
    
    # Ters Euler dönüşümü
    inv_euler = inverse_euler_transform(negated_seq)
    
    # Tekrar negatiflenerek sonuç oluşturuluyor
    transformed = [-a for a in inv_euler]
    
    # İlk eleman (a₀) = 1 olarak sabitleniyor
    transformed[0] = 1
    return transformed

# stripped.csv dosyasını oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Tüm terimlere Lyndon dönüşümünü uygula
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 1:
        continue
    transformed = lyndon_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "LyndonTransformed": transformed
    })

# Dönüştürülmüş diziyi DataFrame'e aktar
df = pd.DataFrame(transformed_sequences)
# Listeyi string'e çevir (dizi olarak kaydedilebilsin diye)
df["LyndonTransformed"] = df["LyndonTransformed"].apply(lambda x: ",".join(map(str, x)))

# CSV'ye kaydet
output_file = "lyndon_transformed_full.csv"
df.to_csv(output_file, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {output_file}")


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Dönüştürülmüş dosya kaydedildi: lyndon_transformed_full.csv


# 26. inverse lyndon transform


/* inverse lyndon transform
 * this function ignores a_0 (A[1]) and assumes it to be 1
 * the output sequence always has b_0=0
 * function uses trv_euler
 */
{ trv_i_lyndon(A) =
  A = -trv_euler(-A);
  A[1] = 0;
  A
}



In [ ]:
import pandas as pd
from sympy import divisors

# Euler Transform
def euler_transform(seq):
    n = len(seq)
    result = []
    for k in range(1, n + 1):
        s = sum((k // d) * seq[d - 1] for d in divisors(k))
        result.append(s)
    return result

# Inverse Lyndon Transform
def inverse_lyndon_transform(seq):
    if not seq:
        return []

    # a₀ = 1 kabul ediliyor
    seq[0] = 1

    # Dizi negatifleniyor
    negated = [-a for a in seq]

    # Euler transformu uygulanıyor
    euler = euler_transform(negated)

    # Sonuç tekrar negatifleniyor
    result = [-a for a in euler]

    # İlk terim 0 olarak ayarlanıyor
    result[0] = 0
    return result

# stripped.csv dosyasını oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Her dizinin tamamına inverse lyndon dönüşümünü uygula
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = inverse_lyndon_transform(original[:])
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseLyndon": transformed
    })

# CSV'ye kaydet
df = pd.DataFrame(transformed_sequences)
output_file = "inverse_lyndon_transformed.csv"
df.to_csv(output_file, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {output_file}")


# 27. moebius transform

/* moebius transform
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=0
 */
{ trv_moebius(A) =
  A=vecextract(A,"2..");
  concat(0,dirdiv(A,vector(#A,n,1)))
}



# 28. inverse moebius transform


/* inverse moebius transform
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=0
 */
{ trv_i_moebius(A) =
  A=vecextract(A,"2..");
  concat(0,dirmul(A,vector(#A,n,1)))
}

Yalnış hesapladı. Aslında kısa sürdü tekrar doğru kodu yazarak çalıştır.

# 29. partial product transform

/* partial product transform
 * b_n = prod {i=0..n} a_n
 */
{ trv_pprod(A) =
  for(i=2,#A,A[i]*=A[i-1]);
  A
}


İşlem çok uzun sürdüğü için dizilerin sadece ilk 20 elemanı dönüştürülmüş ve sonuçlar doğru bulunmuştur. Aslında denemedim belki tüm dizi terimleride çıkar.

In [5]:
import pandas as pd

# Partial product transform function
def partial_product_transform(seq):
    result = []
    product = 1
    for val in seq:
        product *= val
        result.append(product)
    return result

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Partial Product Transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = partial_product_transform(original[:20])
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "PartialProductFirst20": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "partial_product_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {transformed_file_path}")


Dönüştürülmüş dosya kaydedildi: partial_product_transformed.csv


# 30. partial sum transform

/* partial sum transform
 * b_n = sum {i=0..n} a_n
 * inverse of trv_diff
 */
{ trv_psum(A) =
  for(i=2,#A,A[i]+=A[i-1]);
  A
}



İşlem çok uzun sürdüğü için dizilerin sadece ilk 20 elemanı dönüştürülmüş ve sonuçlar doğru bulunmuştur. Aslında denemedim belki tüm dizi terimleride çıkar.

In [6]:
import pandas as pd

# Partial sum transform function
def partial_sum_transform(seq):
    result = []
    total = 0
    for val in seq:
        total += val
        result.append(total)
    return result

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Partial Sum Transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = partial_sum_transform(original[:20])
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "PartialSumFirst20": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "partial_sum_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {transformed_file_path}")


Dönüştürülmüş dosya kaydedildi: partial_sum_transformed.csv


# 31. record values of A 


/* record values of A
 * output sequence b_n is an increasing list of values
 * with b_0 = a_0, b_k, the first value of a>b_{k-1}
 * this function is not invertible
 * output length is determined by the function
 */
{ trv_record(A) = local(B=[A[1]]);
  for(i=2, #A,
    if(A[i]>B[#B], B=concat(B,A[i]));
  );
  B
}


Hepsi doğru çıktı. Dizinin tüm elemanları dönüştürüldü.

In [7]:
import pandas as pd

# Record values transform: keeps increasing values only
def record_values_transform(seq):
    if not seq:
        return []
    result = [seq[0]]
    for val in seq[1:]:
        if val > result[-1]:
            result.append(val)
    return result

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Record Values Transform and store transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if not original:
        continue
    transformed = record_values_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "RecordValues": transformed
    })

# Convert to DataFrame with variable-length sequences as string
transformed_df = pd.DataFrame([{
    "SequenceID": entry["SequenceID"],
    "RecordValues": ",".join(map(str, entry["RecordValues"]))
} for entry in transformed_sequences])

# Save to CSV
transformed_file_path = "record_values_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print(f"Transformed file saved as: {transformed_file_path}")


Transformed file saved as: record_values_transformed.csv


# 32. indices of record values of A



/* indices of record values of A
 * output sequence b_n is indices of a where the record values
 * of trv_record occurred
 * this function is not invertible
 * output length is determined by the function
 */
{ trv_record_ix(A) = local(B=[0],j=1,k=A[1]);
  for(i=2, #A,
    if(A[i]>k,
      B=concat(B,i-1);
      j=i;
      k=A[i];
    );
  );
  B
}

Hepsi doğru çıktı. Dizinin tüm elemanları dönüştürüldü.

In [8]:
import pandas as pd

# Indices of record values transform
def record_indices_transform(seq):
    if not seq:
        return []
    indices = [0]
    current_max = seq[0]
    for i in range(1, len(seq)):
        if seq[i] > current_max:
            indices.append(i)
            current_max = seq[i]
    return indices

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply transform and store results
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if not original:
        continue
    transformed = record_indices_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "RecordIndices": transformed
    })

# Convert to DataFrame with variable-length sequences as string
transformed_df = pd.DataFrame([{
    "SequenceID": entry["SequenceID"],
    "RecordIndices": ",".join(map(str, entry["RecordIndices"]))
} for entry in transformed_sequences])

# Save to CSV
transformed_file_path = "record_indices_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print(f"Transformed file saved as: {transformed_file_path}")


Transformed file saved as: record_indices_transformed.csv


# 33. revert transform

/* revert transform
 * apply reversion of series as ogf
 * this function is its own inverse
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=0
 */
{ trv_revert(A) = local(x='x, B=Ser(A,x)-A[1]);
  B = serreverse(B);
  for(i=1, #A, A[i] = polcoeff(B,i-1));
  A
}



Çok uzun sürüyor ikiye parçala verisetini.

In [ ]:
import pandas as pd
from sympy import symbols, series, Symbol, invert, Rational

# Revert transform for full sequence
def revert_transform(seq):
    if not seq:
        return []

    # Ensure a_0 = 0
    a = [0] + seq[1:] if seq[0] != 0 else seq
    x = Symbol('x')
    
    # Construct the original generating function without the a_0 term
    ogf = sum(Rational(a[i]) * x**i for i in range(1, len(a)))
    
    # Revert the series
    try:
        # Perform series reversion
        reverted = invert(ogf, x, len(a))
        coeffs = [int(reverted.coeff(x, i)) for i in range(len(a))]
        coeffs[0] = 0  # b_0 = 0 by definition
        return coeffs
    except Exception:
        # Return zero list if reversion fails
        return [0] * len(a)

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply revert transform to full sequence lengths
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:  # at least two elements needed
        continue
    transformed = revert_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "RevertTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame([{
    "SequenceID": entry["SequenceID"],
    "RevertTransformed": ",".join(map(str, entry["RevertTransformed"]))
} for entry in transformed_sequences])

output_file = "revert_transformed_full.csv"
transformed_df.to_csv(output_file, index=False)

print(f"Transformed file saved as: {output_file}")


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 34. revert_egf transform


/* revert_egf transform
 * apply reversion of series as egf
 * this function is its own inverse
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=0
 */
{ trv_revert_egf(A) = local(x='x,B);
  B = sum(i=1, #A-1, A[i+1]*x^i/i!);
  B = serreverse(B+O(x^#A));
  for(i=0,#A-1, A[i+1] = polcoeff(B,i)*i!);
  A
}


Çok uzun sürüyor ikiye parçala verisetini.

In [ ]:
import pandas as pd
from sympy import symbols, series, invert, Rational, factorial, Symbol

# Revert EGF transform function
def revert_egf_transform(seq):
    if not seq:
        return []

    # Ensure a_0 = 0 by convention
    a = [0] + seq[1:] if seq[0] != 0 else seq
    x = Symbol('x')

    # Build EGF: sum a[n+1] * x^n / n!
    egf = sum(Rational(a[i + 1]) * x**i / factorial(i) for i in range(len(a) - 1))

    try:
        # Perform reversion of the EGF series
        reversed_series = invert(egf, x, len(a))

        # Extract coefficients and multiply by factorials to get sequence back from EGF
        reverted = [0] + [int(reversed_series.coeff(x, i) * factorial(i)) for i in range(1, len(a))]
        return reverted
    except Exception:
        # If inversion fails, return zeros
        return [0] * len(a)

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply revert_egf transform to all sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = revert_egf_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "RevertEGF": transformed
    })

# Save results to CSV
transformed_df = pd.DataFrame([{
    "SequenceID": entry["SequenceID"],
    "RevertEGF": ",".join(map(str, entry["RevertEGF"]))
} for entry in transformed_sequences])

output_file = "revert_egf_transformed.csv"
transformed_df.to_csv(output_file, index=False)

print(f"Transformed file saved as: {output_file}")


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 35. stirling transform

/* stirling transform
 * apply given labeled structure to non-empty sets
 */
{ trv_stirling(A) = local(x='x,B);
  B = sum(i=0, #A-1, A[i+1]*x^i/i!);
  B = subst(B+O(x^#A),x,exp(x+O(x^#A))-1);
  for(i=0,#A-1, A[i+1] = polcoeff(B,i)*i!);
  A
}



Çok uzun sürüyor ikiye parçala verisetini.

In [ ]:
import pandas as pd
from sympy import symbols, exp, factorial, Rational, series

# Stirling transform function
def stirling_transform(seq):
    if not seq:
        return []

    x = symbols('x')
    n = len(seq)

    # Build EGF: sum A[n] * x^n / n!
    egf = sum(Rational(seq[i]) * x**i / factorial(i) for i in range(n))

    # Substitute x -> exp(x) - 1
    substituted = egf.subs(x, exp(x) - 1).series(x, 0, n)

    # Extract coefficients and multiply by factorial to get integer sequence
    transformed = [int(substituted.coeff(x, i) * factorial(i)) for i in range(n)]
    return transformed

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Stirling transform to all sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = stirling_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "StirlingTransform": transformed
    })

# Save results to CSV
transformed_df = pd.DataFrame([{
    "SequenceID": entry["SequenceID"],
    "StirlingTransform": ",".join(map(str, entry["StirlingTransform"]))
} for entry in transformed_sequences])

output_file = "stirling_transformed.csv"
transformed_df.to_csv(output_file, index=False)

print(f"Transformed file saved as: {output_file}")


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 36. inverse stirling transform

/* inverse stirling transform
 */
{ trv_i_stirling(A) = local(x='x,B);
  B = sum(i=0, #A-1, A[i+1]*x^i/i!);
  B = subst(B+O(x^#A),x,log(1+x+O(x^#A)));
  for(i=0,#A-1, A[i+1] = polcoeff(B,i)*i!);
  A
}


Çok uzun sürüyor ikiye parçala verisetini.

In [ ]:
import pandas as pd
from sympy import symbols, log, factorial, Rational, series

# Inverse Stirling transform function
def inverse_stirling_transform(seq):
    if not seq:
        return []

    x = symbols('x')
    n = len(seq)

    # Construct the exponential generating function
    egf = sum(Rational(seq[i]) * x**i / factorial(i) for i in range(n))

    # Apply the substitution x -> log(1 + x)
    substituted = egf.subs(x, log(1 + x)).series(x, 0, n)

    # Convert back to integer sequence
    transformed = [int(substituted.coeff(x, i) * factorial(i)) for i in range(n)]
    return transformed

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse Stirling transform to all sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = inverse_stirling_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseStirlingTransform": transformed
    })

# Save results to CSV
transformed_df = pd.DataFrame([{
    "SequenceID": entry["SequenceID"],
    "InverseStirlingTransform": ",".join(map(str, entry["InverseStirlingTransform"]))
} for entry in transformed_sequences])

output_file = "inverse_stirling_transformed.csv"
transformed_df.to_csv(output_file, index=False)

print(f"Transformed file saved as: {output_file}")


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 37. weigh transform

/* weigh transform
 * apply set structure to sequence of unlabeled objects
 * injectively (or asymmetrically)
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 * trv_weigh_x is a version not making that assumption
 */
{ trv_weigh(A) = local(i=-1,B=vector(#A-1,n,i=-i;i/n),C);
  B = dirmul(vecextract(A,"2.."),B);
  C = exp(Ser(concat(0,B)));
  for(i=1, #A, A[i] = polcoeff(C,i-1));
  A
}




Çok uzun sürüyor ikiye parçala verisetini.

In [ ]:
import pandas as pd
from sympy import symbols, exp, factorial, Rational, series

# Weigh transform function
def weigh_transform(seq):
    if len(seq) < 2:
        return []

    n = len(seq)
    x = symbols('x')

    # Step 1: Drop a₀ and treat A[1:] as the input
    A = seq[1:]

    # Step 2: Construct the Dirichlet-multiplied vector B with 1/n factors
    B = [Rational(A[i] // 1, i + 1) for i in range(n - 1)]  # i from 1 to n-1

    # Step 3: Create exponential generating function
    C = exp(sum(B[i] * x**(i + 1) for i in range(len(B)))).series(x, 0, n)

    # Step 4: Extract coefficients and build full sequence (assume a₀ = 0, set b₀ = 1)
    result = [1]  # b₀ = 1
    result.extend([int(C.coeff(x, i)) for i in range(1, n)])

    return result

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply weigh transform to all sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = weigh_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "WeighTransform": transformed
    })

# Save results to CSV
transformed_df = pd.DataFrame([{
    "SequenceID": entry["SequenceID"],
    "WeighTransform": ",".join(map(str, entry["WeighTransform"]))
} for entry in transformed_sequences])

output_file = "weigh_transformed.csv"
transformed_df.to_csv(output_file, index=False)

print(f"Transformed file saved as: {output_file}")


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 38. inverse weigh transform

/* inverse weigh transform
 * this function ignores a_0 (A[1]) and assumes it to be 1
 * the output sequence always has b_0=0
 * trv_i_weigh_x is a version not making that assumption
 */
{ trv_i_weigh(A) = local(i=-1,B=vector(#A-1,n,i=-i;i*1/n),C);
  A[1] = 1;
  C = log(Ser(A));
  A=vecextract(A,"2..");
  for(i=1, #A, A[i] = polcoeff(C,i));
  A = dirdiv(A,B);
  concat(0,A)
}




Çok uzun sürüyor ikiye parçala verisetini.

In [ ]:
import pandas as pd
import numpy as np
from sympy import symbols, log, series, expand, Poly

# Inverse Weigh Transform function
def inverse_weigh_transform(seq):
    n = len(seq)
    if n < 2:
        return []

    # Step 1: Assume a_0 = 1 and update the sequence
    A = [1] + seq[1:n]  # a_0 = 1, geri kalan ilk n-1 terim alınır

    # Step 2: Compute the logarithm of the generating function
    x = symbols('x')
    fx = sum(A[i] * x**i for i in range(n))
    log_series = series(log(fx), x, 0, n).removeO()

    # Step 3: Extract coefficients of log(f(x))
    poly = Poly(expand(log_series), x)
    coeffs = [poly.coeff_monomial(x**i) for i in range(1, n)]  # i = 1 to n-1

    # Step 4: Apply inverse transformation
    transformed = [0] + [float(coeffs[i - 1]) / i for i in range(1, n)]
    
    return transformed

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse weigh transform to all sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = inverse_weigh_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseWeighTransform": transformed
    })

# Save results to CSV
df = pd.DataFrame([{
    "SequenceID": entry["SequenceID"],
    "InverseWeighTransform": ",".join(f"{x:.10g}" for x in entry["InverseWeighTransform"])
} for entry in transformed_sequences])

output_file = "inverse_weigh_transformed.csv"
df.to_csv(output_file, index=False)

print(f"Transformed file saved as: {output_file}")


C:\Users\ÇELİK\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 39. weigh labeled transform



/* weigh labeled transform
 * apply set structure to sequence of labeled objects
 * injectively (or asymmetrically for L-species)
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 * function uses trv_exp
 */
{ trv_weigh_l(A) = local(B=vector(#A-1),i,s);
  A=vecextract(A,"2..");
  for(k=1, #A,
    s=1;
    i=1;
    forstep(j=k,#A,k,
      if(i>1, s*=(1-i)*binomial(j-1,j-k));
      B[j] += s*A[k];
      i++;
    );
  );
  trv_exp(concat(0,B))
}

In [5]:
import pandas as pd
import numpy as np
from math import comb
from decimal import Decimal, getcontext

# Ondalık hassasiyet arttırılıyor
getcontext().prec = 100

# Exponential transform function
def trv_exp(seq):
    n = len(seq)
    B = [Decimal(1)]
    for i in range(n):
        B.append(B[-1] * Decimal(seq[i]) / Decimal(i + 1))
    return [float(x) for x in B[1:]]

# Weigh labeled transform function (ilk 20 elemana sınırlı)
def weigh_labeled_transform(seq):
    seq = seq[:20]  # 🔒 Diziyi ilk 20 elemana sınırla
    n = len(seq)
    if n < 2:
        return []

    B = [Decimal(0)] * (n - 1)
    A = [Decimal(x) for x in seq[1:]]

    for k in range(1, n):
        s = Decimal(1)
        i = 1
        for j in range(k, n, k):
            if i > 1:
                s *= (1 - i) * Decimal(comb(j - 1, j - k))
            B[j - 1] += s * A[k - 1]
            i += 1

    result = trv_exp([0] + B)  # a₀ = 0
    return result

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply transformation
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = weigh_labeled_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "WeighLabeledTransformFirst20": transformed
    })

# Save to CSV
df = pd.DataFrame([{
    "SequenceID": entry["SequenceID"],
    "WeighLabeledTransformFirst20": ",".join(f"{x:.10g}" for x in entry["WeighLabeledTransformFirst20"])
} for entry in transformed_sequences])

output_file = "weigh_labeled_transformed_first20.csv"
df.to_csv(output_file, index=False)

print(f"İlk 20 elemana dönüşüm uygulandı. Dosya kaydedildi: {output_file}")


İlk 20 elemana dönüşüm uygulandı. Dosya kaydedildi: weigh_labeled_transformed_first20.csv


# 40. inverse weigh labeled transform

/* inverse weigh labeled transform
 * this function ignores a_0 (A[1]) and assumes it to be 1
 * the output sequence always has b_0=0
 * function uses trv_log
 */
{ trv_i_weigh_l(A) = local(i,s);
  A=trv_log(A);
  A=vecextract(A,"2..");
  for(k=1, #A\2,
    s=1;
    i=2;
    forstep(j=k+k,#A,k,
      s*=(1-i)*binomial(j-1,j-k);
      A[j] -= s*A[k];
      i++;
    );
  );
  concat(0,A)
}

# 41. egf to sequence

/* egf to sequence
 * output vector of sequence whose exponential generating function
 * is given by the input series ("t_SER")
 */
{ trsv_egf(A) = vector(#A, i, polcoeff(A,i-1)*(i-1)!)
}



# 42. ogf multiplication operation

/* ogf multiplication operation
 * multiplies ogf of sequences
 * sequence length is the shorter of the 2
 */
{ opv_mul_ogf(A, B) = local(C=Ser(A,x)*Ser(B,x));
  A=vector(min(#A,#B));
  for(i=1, #A, A[i] = polcoeff(C,i-1));
  A
}


# 43. ogf division operation

/* ogf division operation
 * divides ogf of sequences
 * sequence length is the shorter of the 2
 */
{ opv_div_ogf(A, B) = local(C=Ser(A,x)/Ser(B,x));
  A=vector(min(#A,#B));
  for(i=1, #A, A[i] = polcoeff(C,i-1));
  A
}

# 44. egf multiplication operation

/* egf multiplication operation
 * multiplies egf of sequences
 * sequence length is the shorter of the 2
 */
{ opv_mul_egf(A, B) = local(C=vector(min(#A,#B)));
  forstep(i=#C-1,0,-1,C[i+1]=sum(j=0,i,binomial(i,j)*A[j+1])*B[i-j+1]);
  C
}

In [23]:
import pandas as pd
from math import comb

# EGF multiplication function
def opv_mul_egf(seq_a, seq_b):
    n = min(len(seq_a), len(seq_b))  # Take the shorter length
    C = [0] * n  # Initialize the result sequence
    
    # Compute the convolution for the product of the EGFs
    for i in range(n):
        C[i] = sum(comb(i, j) * seq_a[j] * seq_b[i - j] for j in range(i + 1))
    
    return C

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply EGF multiplication and store only the transformed sequences
transformed_sequences = []
for i in range(len(sequences) - 1):
    seq_a = sequences[i]["Sequence"]
    seq_b = sequences[i + 1]["Sequence"]
    
    # Apply the EGF multiplication only if both sequences have at least one element
    if len(seq_a) > 0 and len(seq_b) > 0:
        transformed = opv_mul_egf(seq_a, seq_b)
        transformed_sequences.append({
            "SequenceID_A": sequences[i]["SequenceID"],
            "SequenceID_B": sequences[i + 1]["SequenceID"],
            "EGF_Multiplied": transformed
        })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "egf_multiplied_sequences.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


'egf_multiplied_sequences.csv'

# 45. egf:dgf multiplication operation

/* egf:dgf multiplication operation
 * returns sequence whose egf is b_1*A(x) + b_2*A(x^2/2!) + ...
 * sequence length is the shorter of the 2
 * note that this operation is not commutative
 */
{ opv_mul_ed(A, B) = local(C,i,s);
  C=vector(min(#A,#B));
  for(k=1, #C-1,
    s=1;
    i=1;
    forstep(j=k,#C-1,k,
      s*=binomial(j-1,j-k);
      C[j+1] += s*A[i+1]*B[k+1];
      i++;
    );
  );
  C
}



In [22]:
import pandas as pd
from math import comb

# EGF:DGf multiplication function
def opv_mul_ed(seq_a, seq_b):
    n = min(len(seq_a), len(seq_b))  # Take the shorter length
    C = [0] * n  # Initialize the result sequence
    
    # Compute the multiplication operation as described
    for k in range(1, n):  # Loop over the sequence
        s = 1
        i = 1
        for j in range(k, n, k):  # Step through in k increments
            s *= comb(j - 1, j - k)  # Binomial coefficient
            C[j] += s * seq_a[i] * seq_b[k]  # Apply the transformation
            i += 1
    
    return C

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply EGF:DGf multiplication and store only the transformed sequences
transformed_sequences = []
for i in range(len(sequences) - 1):
    seq_a = sequences[i]["Sequence"]
    seq_b = sequences[i + 1]["Sequence"]
    
    # Apply the EGF:DGf multiplication only if both sequences have at least one element
    if len(seq_a) > 0 and len(seq_b) > 0:
        transformed = opv_mul_ed(seq_a, seq_b)
        transformed_sequences.append({
            "SequenceID_A": sequences[i]["SequenceID"],
            "SequenceID_B": sequences[i + 1]["SequenceID"],
            "EGF_DGF_Multiplied": transformed
        })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "egf_dgf_multiplied_sequences.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


'egf_dgf_multiplied_sequences.csv'

# 46. boustrophedon transform

/* boustrophedon transform
 */
{ trv_bous(A) = local(x='x,B);
  B = sum(i=0, #A-1, A[i+1]*x^i/i!);
  B=(tan(x+O(x^#A))+1/cos(x+O(x^#A)))*(B+O(x^#A));
  for(i=0,#A-1, A[i+1] = polcoeff(B,i)*i!);
  A
}



Çok uzun sürüyor ikiye parçala verisetini.

In [ ]:
import pandas as pd
import sympy as sp
from math import factorial

# Boustrophedon transform function
def boustrophedon_transform(seq):
    n = len(seq)
    x = sp.symbols('x')
    
    # Step 1: Compute the EGF (exponential generating function)
    egf = sum(seq[i] * x**i / factorial(i) for i in range(n))
    
    # Step 2: Compute tan(x) + sec(x) up to n terms using sympy.series (without O-terms)
    T = sp.series(sp.tan(x), x, n=n).removeO()
    S = sp.series(1 / sp.cos(x), x, n=n).removeO()
    
    transform_function = sp.series((T + S) * egf, x, n=n).removeO()
    
    # Step 3: Extract coefficients
    coeffs = transform_function.as_poly(x).all_coeffs()
    coeffs = coeffs[::-1]  # sympy returns highest degree first
    
    # Convert EGF back to integer sequence by multiplying with factorials
    return [int(sp.simplify(coeffs[i] * factorial(i))) for i in range(len(coeffs))]

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Boustrophedon transform and store the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    try:
        transformed = boustrophedon_transform(original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "BoustrophedonTransformed": transformed
        })
    except Exception as e:
        print(f"Hata (ID: {entry['SequenceID']}): {e}")

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_df["BoustrophedonTransformed"] = transformed_df["BoustrophedonTransformed"].apply(lambda x: ", ".join(map(str, x)))
transformed_file_path = "boustrophedon_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("Dönüştürülmüş dosya kaydedildi:", transformed_file_path)


# 47. inverse boustrophedon transform

/* inverse boustrophedon transform
 */
{ trv_i_bous(A) = local(x='x,B);
  B = sum(i=0, #A-1, A[i+1]*x^i/i!);
  B=(B+O(x^#A))/(tan(x+O(x^#A))+1/cos(x+O(x^#A)));
  for(i=0,#A-1, A[i+1] = polcoeff(B,i)*i!);
  A
}





Çok uzun sürüyor ikiye parçala verisetini.

In [ ]:
import pandas as pd
from math import factorial
import sympy as sp

# Inverse Boustrophedon transform function
def inverse_boustrophedon_transform(seq):
    n = len(seq)
    x = sp.symbols('x')
    
    # Step 1: Compute the exponential generating function of the sequence
    sum_series = sum(seq[i] * x**i / factorial(i) for i in range(n))

    # Step 2: Compute the asymptotic expansion of tan(x) + sec(x)
    tan_series = sp.series(sp.tan(x), x, 0, n)
    sec_series = sp.series(1 / sp.cos(x), x, 0, n)
    denom_series = sp.series(tan_series.removeO() + sec_series.removeO(), x, 0, n)

    # Step 3: Compute the division (EGF / (tan(x) + sec(x)))
    B_series = sp.series(sum_series / denom_series, x, 0, n).removeO()

    # Step 4: Extract coefficients and return ordinary sequence
    coeffs = [sp.simplify(B_series.coeff(x, i) * factorial(i)) for i in range(n)]
    return [int(coeff) for coeff in coeffs]

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse Boustrophedon transform
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 20:
        continue
    try:
        transformed = inverse_boustrophedon_transform(original[:20])
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "InverseBoustrophedonTransformed": transformed
        })
    except Exception as e:
        print(f"Error processing {entry['SequenceID']}: {e}")
        continue

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "inverse_boustrophedon_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print(f"Transformed sequences saved to {transformed_file_path}")


# 48. carlitz transform

/* carlitz transform
 * apply list structure to sequence of unlabeled objects
 * insisting that neighbors be different object types
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 * trv_carlitz_x is a version not making that assumption
 * function uses trv_invert
 */
{ trv_carlitz(A) = local(i=-1,B=vector(#A-1,n,i=-i;i));
  A = concat(0,dirmul(vecextract(A,"2.."),B));
  trv_invert(A)
}

Sonçlar yalnış çıkıyor. Tekrar kodu yaz.

# 49. inverse carlitz transform

/* inverse carlitz transform
 * this function ignores a_0 (A[1]) and assumes it to be 1
 * the output sequence always has b_0=0
 * function uses trv_i_invert
 */
{ trv_i_carlitz(A) = local(i=-1,B=vector(#A-1,n,i=-i;i),C);
  A = trv_i_invert(A);
  concat(0,dirdiv(vecextract(A,"2.."),B));
}


Sonçlar yalnış çıkıyor. Tekrar kodu yaz.

# 50. extended version of carlitz transform

/* extended version of carlitz transform
 * it does not assume a_0 = A[1] == 0
 * function uses trv_invert_x
 */
{ trv_carlitz_x(A) = local(i=-1,B=vector(#A-1,n,i=-i;i));
  A = concat(A[1]/2,dirmul(vecextract(A,"2.."),B));
  trv_invert_x(A)
}



Sonçlar yalnış çıkıyor. Tekrar kodu yaz.

# 51. chain transform
/* chain transform
 * apply chain (undirected list) structure to sequence of
 * unlabeled objects
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 */
{ trv_chain(A) = local(x='x, B=Ser(A,x)-A[1], B2 = subst(B,x,x^2));
  B = (2-B^2)/(2*(1-B)) + B2*(1+B)/(2*(1-B2));
  for(i=1, #A, A[i] = polcoeff(B,i-1));
  A
}



Sonçlar yalnış çıkıyor. Tekrar kodu yaz.

# 52. chain transform for matrices

/* chain transform for matrices
 * this function ignores a_0,0 (A[1,1]) and assumes it to be 0
 * the output sequence always has b_0,0=1
 */
{ trm_chain(A) =
  local(x='x,y='y,m=matsize(A)[1],n=matsize(A)[2],B=O(x^m)+O(y^n),
    B2=O(x^m)+O(y^n),j=1);
  for(i=0, m-1,
    for(j=j, n-1, B+=A[i+1,j+1]*x^i*y^j);
    j=0;
  );
  for(i=0, (m-1)\2,
    for(j=j, (n-1)\2, B2+=A[i+1,j+1]*x^(2*i)*y^(2*j));
    j=0;
  );
  B = (2-B^2)/(2*(1-B)) + B2*(1+B)/(2*(1-B2));
  for(i=0, m-1,
    for(j=0, n-1, A[i+1,j+1] = polcoeff(polcoeff(B,j,y),i,x));
  );
  A
}



Sonçlar yalnış çıkıyor. Tekrar kodu yaz.


# 53. chain injective transform

/* chain injective transform
 * apply chain (undirected list) structure to sequence of
 * unlabeled objects injectively (or asymmetrically)
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 */
{ trv_chain_j(A) = local(x='x, B=Ser(A,x)-A[1]);
  B2 = subst(B,x,x^2);
  B = (2-B^2)/(2*(1-B)) - B2*(1+B)/(2*(1-B2));
  for(i=1, #A, A[i] = polcoeff(B,i-1));
  A
}


Sonçlar yalnış çıkıyor. Tekrar kodu yaz.


# 54. chain injective transform for matrices

/* chain injective transform for matrices
 * this function ignores a_0,0 (A[1,1]) and assumes it to be 0
 * the output sequence always has b_0,0=1
 */
{ trm_chain_j(A) =
  local(x='x,y='y,m=matsize(A)[1],n=matsize(A)[2],B=O(x^m)+O(y^n),
    B2=O(x^m)+O(y^n),j=1);
  for(i=0, m-1,
    for(j=j, n-1, B+=A[i+1,j+1]*x^i*y^j);
    j=0;
  );
  for(i=0, (m-1)\2,
    for(j=j, (n-1)\2, B2+=A[i+1,j+1]*x^(2*i)*y^(2*j));
    j=0;
  );
  B = (2-B^2)/(2*(1-B)) - B2*(1+B)/(2*(1-B2));
  for(i=0, m-1,
    for(j=0, n-1, A[i+1,j+1] = polcoeff(polcoeff(B,j,y),i,x));
  );
  A
}

In [ ]:
import pandas as pd
import numpy as np

# Helper function: substitute x -> x^2 (not needed for matrix version but retained for compatibility)
def subst(seq, x, x_squared):
    return [val * (x_squared ** i) for i, val in enumerate(seq)]

# Chain injective transform for matrices
def chain_injective_transform_matrix(A):
    m, n = A.shape
    x = 1  # Assuming x = 1
    y = 1  # Assuming y = 1

    B = np.zeros((m, n))
    B2 = np.zeros((m, n))
    
    # Fill B
    for i in range(m):
        for j in range(n):
            B[i, j] = A[i, j] * (x ** i) * (y ** j)
    
    # Fill B2 with x^2, y^2 substitutions
    for i in range(m):
        for j in range(n):
            B2[i, j] = A[i, j] * (x ** (2 * i)) * (y ** (2 * j))
    
    # Avoid division by zero
    with np.errstate(divide='ignore', invalid='ignore'):
        denom1 = 2 * (1 - B)
        denom2 = 2 * (1 - B2)
        part1 = np.divide((2 - B ** 2), denom1, where=denom1 != 0)
        part2 = np.divide(B2 * (1 + B), denom2, where=denom2 != 0)
        B_transformed = np.where((denom1 == 0) | (denom2 == 0), 0, part1 - part2)

    return B_transformed

# Read stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply transformation
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    length = len(original)
    matrix_size = int(length ** 0.5)

    if matrix_size * matrix_size != length:
        # Not a perfect square, skip
        continue

    try:
        matrix = np.array(original).reshape((matrix_size, matrix_size))
        transformed_matrix = chain_injective_transform_matrix(matrix)
        flattened_transformed = transformed_matrix.flatten().tolist()

        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "ChainInjectiveTransformed": flattened_transformed
        })
    except Exception as e:
        print(f"Error processing {entry['SequenceID']}: {e}")

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_df["ChainInjectiveTransformed"] = transformed_df["ChainInjectiveTransformed"].apply(lambda x: ", ".join(map(str, x)))
output_path = "chain_injective_transformed_matrices.csv"
transformed_df.to_csv(output_path, index=False)

print(f"Transformed matrices saved to: {output_path}")


# 55. chain labeled transform

/* chain labeled transform
 * apply chain (undirected list) structure to sequence of labeled objects
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 */
{ trv_chain_l(A) = local(x='x, B);
  B = sum(i=1, #A-1, A[i+1]*x^i/i!);
  B = (1/(1-B+O(x^#A))+1+B)/2;
  for(i=0,#A-1, A[i+1] = polcoeff(B,i)*i!);
  A
}




Sonçlar yalnış çıkıyor. Tekrar kodu yaz.


# 56. inverse chain labeled transform

/* inverse chain labeled transform
 * this function ignores a_0 (A[1]) and assumes it to be 1
 * the output sequence always has b_0=0
 */
{ trv_i_chain_l(A) = local(x='x, B);
  B = sum(i=1, #A-1, A[i+1]*x^i/i!);
  B = 1+B-sqrt(1+B^2+O(x^#A));
  for(i=0,#A-1, A[i+1] = polcoeff(B,i)*i!);
  A
}


In [18]:
import pandas as pd
import numpy as np
from scipy.special import factorial

# Inverse chain labeled transform function for entire sequence
def inverse_chain_labeled_transform(seq):
    n = len(seq)
    x = 1  # Assume x = 1 for simplicity

    # Compute B from sequence (skip seq[0])
    B = 0.0
    for i in range(1, n):
        try:
            B += seq[i] * (x ** i) / factorial(i, exact=False)
        except OverflowError:
            B += 0.0

    # Apply inverse transformation formula
    try:
        B_trans = 1 + B - np.sqrt(1 + B ** 2)
    except (OverflowError, ValueError):
        B_trans = 0.0  # Fallback if math domain error

    # Build new sequence from B_trans
    new_seq = []
    for i in range(n):
        try:
            val = B_trans * (x ** i) / factorial(i, exact=False)
            new_seq.append(float(val))
        except OverflowError:
            new_seq.append(0.0)

    return new_seq

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse chain labeled transform
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    try:
        transformed = inverse_chain_labeled_transform(original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "InverseChainLabeledTransformed": transformed
        })
    except Exception as e:
        print(f"Error in sequence {entry['SequenceID']}: {e}")

# Save results to CSV
transformed_df = pd.DataFrame(transformed_sequences)
# Convert list to string for CSV
transformed_df["InverseChainLabeledTransformed"] = transformed_df["InverseChainLabeledTransformed"].apply(
    lambda x: ", ".join(map(str, x))
)
output_path = "inverse_chain_labeled_transformed_all.csv"
transformed_df.to_csv(output_path, index=False)

print(f"Transformed sequences saved to: {output_path}")


C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_10612\1946025634.py:20: RuntimeWarning: overflow encountered in scalar power
  B_trans = 1 + B - np.sqrt(1 + B ** 2)


Transformed sequences saved to: inverse_chain_labeled_transformed_all.csv


# 57. chain "boy-girl" transform

/* chain "boy-girl" transform
 * apply chain (undirected list) structure to sequence of
 * unlabeled objects
 * insisting that neighbors be different object types
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 * function uses trv_carlitz, opv_mul_ogf
 */
{ trv_chain_bg(A) = local(B,C,D=[0,1,0,-1]);
  B = trv_carlitz(A);
  C = dirmul(vecextract(B,"2.."), vector(#B-1,n,n==2));
  A = vecextract(A,"2..");
  A = dirmul(A, vector(#A,n,D[1+bitand(n,3)]));
  A = opv_mul_ogf(concat(0,A), concat(1,C));
  A[1] = 1;
  (A+B)/2
}

Sonçlar yalnış çıkıyor. Tekrar kodu yaz.


# 58. chebyshev transform


/* chebyshev transform
 * G.f. B(x) = (1-x^2)/(1+x^2)*A(x/(1+x^2))
 * Used by Paul Barry for A100047
 */
{ trv_chebyshev(A) = local(x='x, B=Ser(A,x));
  B = subst(B,x,x/(1+x^2+O(x^#A)))*(1-x^2+O(x^#A))/(1+x^2+O(x^#A));
  for(i=1, #A, A[i] = polcoeff(B,i-1));
  A
}


Sonçlar yalnış çıkıyor. Tekrar kodu yaz.

# 59. inverse chebyshev transform

/* inverse chebyshev transform
 */
{ trv_i_chebyshev(A) = local(x='x, B=Ser(A,x));
  B = subst(B*(1+x^2+O(x^#A))/(1-x^2+O(x^#A)),x,serreverse(x/(1+x^2+O(x^#A))));
  for(i=1, #A, A[i] = polcoeff(B,i-1));
  A
}

In [ ]:
import pandas as pd
from sympy import symbols, series, expand, simplify
from sympy.abc import x

# Manuel seri terslemesi (x / (1 + x^2)'nin ters serisi)
def manual_reversion(f, x, order):
    g = x  # initial guess g(x) = x
    for _ in range(order):
        g = simplify(g - (f.subs(x, g) - x))  # Newton-Raphson-like iteration
        g = g.series(x, 0, order).removeO()
    return g

# Inverse Chebyshev Transform function
def inverse_chebyshev_transform(seq):
    n = len(seq)
    
    # Generating function A(x)
    A = sum(seq[i] * x**i for i in range(n))
    A = A.series(x, 0, n).removeO()

    # Original substitution
    subst_expr = x / (1 + x**2)

    # Manually compute inverse of substitution
    inverse_subst = manual_reversion(subst_expr, x, n)

    # Apply inverse Chebyshev transform
    B = A * (1 + x**2) / (1 - x**2)
    B = B.series(x, 0, n).removeO()
    B = B.subs(x, inverse_subst).series(x, 0, n).removeO()

    # Extract integer coefficients
    transformed = [int(B.coeff(x, i)) for i in range(n)]
    return transformed

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse Chebyshev transform
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 5:
        continue
    try:
        transformed = inverse_chebyshev_transform(original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "InverseChebyshev": transformed
        })
    except Exception as e:
        print(f"Error in {entry['SequenceID']}: {e}")

# Save to CSV
df = pd.DataFrame(transformed_sequences)
df["InverseChebyshev"] = df["InverseChebyshev"].apply(lambda seq: ", ".join(map(str, seq)))
output_path = "inverse_chebyshev_transformed_full.csv"
df.to_csv(output_path, index=False)

print(f"Ters dönüşüm başarıyla tamamlandı: {output_path}")


# 60. continuant transform

/* continuant transform
 */
{ trv_continuant(A) = local(B=vector(#A+1));
  B[1]=1;
  B[2]=A[1];
  for(i=2,#A, B[i+1]=A[i]*B[i]+A[i-1]);
  B
}


In [1]:
import pandas as pd

# Continuant transform function
def continuant_transform(seq):
    n = len(seq)
    if n == 0:
        return []
    B = [1]
    if n >= 1:
        B.append(seq[0])
    for i in range(1, n):
        next_val = seq[i] * B[i] + seq[i - 1]
        B.append(next_val)
    return B

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply continuant transform to all sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue  # Need at least 2 elements for meaningful transform
    transformed = continuant_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "ContinuantTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "continuant_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


'continuant_transformed.csv'

# 61. inverse continuant transform

/* inverse continuant transform
 */
{ trv_i_continuant(A) = local(B=vector(#A-1));
  A[1]=1;
  B[1]=A[2];
  for(i=2,#A-1, B[i]=(A[i+1]-B[i-1])/A[i]);
  B
}


In [1]:
import pandas as pd
from fractions import Fraction

# Inverse continuant transform using rational arithmetic (GP-compatible)
def inverse_continuant_transform(seq):
    if len(seq) < 2:
        return []
    A = seq.copy()
    A[0] = 1  # Set A[1] = 1 in 1-based indexing
    B = [Fraction(A[1])]
    for i in range(2, len(A)):
        numerator = Fraction(A[i]) - B[i - 2]
        denominator = Fraction(A[i - 1])
        if denominator == 0:
            B.append(Fraction(0))  # Or handle as needed
        else:
            B.append(numerator / denominator)
    return B

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply the transform to all sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 3:
        continue  # At least 3 elements are needed
    transformed = inverse_continuant_transform(original)
    transformed_str = "[" + ", ".join(str(x) for x in transformed) + "]"
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseContinuantTransformed": transformed_str
    })

# Save result to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "inverse_continuant_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print(f"Saved transformed sequences to {transformed_file_path}")


Saved transformed sequences to inverse_continuant_transformed.csv


# 62. cycle transform for matrices

/* cycle transform for matrices
 * this function ignores a_0,0 (A[1,1]) and assumes it to be 0
 * the output sequence always has b_0,0=1
 * function uses trm_i_moebius and trm_lyndon
 * function indirectly uses trm_i_euler
 */
{ trm_cycle(A) =
  A = trm_i_moebius(trm_lyndon(A));
  A[1,1] = 1;
  A
}

In [2]:
import pandas as pd

# Placeholder for Lyndon transform (currently returns a copy)
def trm_lyndon(seq):
    # TODO: Replace with actual Lyndon transform logic
    return seq[:]  # Copy of the sequence

# Placeholder for Möbius transform (currently returns a copy)
def trm_i_moebius(seq):
    # TODO: Replace with actual Möbius transform logic
    return seq[:]  # Copy of the sequence

# Cycle transform function for sequences
def trm_cycle(seq):
    A = trm_lyndon(seq)
    A = trm_i_moebius(A)
    if A:
        A[0] = 1  # Force b_0,0 = 1 as per specification
    return A

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as file:
    for line in file:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue  # Skip lines with non-integer values

# Apply cycle transform to each sequence
transformed_sequences = []
for entry in sequences:
    original_seq = entry["Sequence"]
    transformed_seq = trm_cycle(original_seq)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "CycleTransformed": transformed_seq
    })

# Save the transformed sequences to CSV
df = pd.DataFrame(transformed_sequences)
output_file = "cycle_transformed_for_metrics.csv"
df.to_csv(output_file, index=False)
print(f"Transformed file saved as: {output_file}")


Transformed file saved as: cycle_transformed_for_metrics.csv


# 63. inverse cycle transform for matrices

/* inverse cycle transform for matrices
 * this function ignores a_0,0 (A[1,1]) and assumes it to be 1
 * the output sequence always has b_0,0=0
 * function uses trm_moebius and trm_i_lyndon
 * function indirectly uses trm_euler
 */
{ trm_i_cycle(A) = trm_i_lyndon(trm_moebius(A));
}



Sonçlar yalnış çıkıyor. Tekrar kodu yaz.


# 64. cycle product transform

/* cycle product transform
 * apply cycle structure to sequence of unlabeled objects
 * to assemble them by the product of their sizes
 * this function ignores a_0, a_1 (A[0], A[1]) and assumes them to be 0
 * the output sequence always has b_0=0, b_1=1
 * function uses trv_i_moebius_p and trv_lyndon_p
 * function indirectly uses trv_i_euler_p
 */
{ trv_cycle_p(A) =
  A = trv_i_moebius_p(trv_lyndon_p(A));
  A[2] = 1;
  A
}


Sonçlar yalnış çıkıyor. Tekrar kodu yaz.


# 65. inverse cycle product transform

/* inverse cycle product transform
 * this function ignores a_0, a_1 (A[0], A[1]) and assumes them to be 0, 1
 * the output sequence always has b_0=b_1=0
 * function uses trv_moebius_p and trv_i_lyndon_p
 * function indirectly uses trv_euler_p
 */
{ trv_i_cycle_p(A) = trv_i_lyndon_p(trv_moebius_p(A));
}



aşağıdaki kod yakın buluyor biraz sıkıntılar var. Aslında çoğu doğru. Analiz uzun sürüyor.

In [2]:
import pandas as pd
from sympy import mobius, divisors, totient
from fractions import Fraction

# --- Möbius transform for product structures ---
def moebius_p_transform(seq):
    n = len(seq)
    result = [Fraction(0) for _ in range(n)]
    for k in range(1, n + 1):
        result[k - 1] = sum(
            Fraction(mobius(k // d)) * seq[d - 1]
            for d in divisors(k)
        )
    return result

# --- Inverse Lyndon transform (same as trv_i_lyndon_p) ---
def inverse_lyndon_p_transform(seq):
    n = len(seq)
    result = [Fraction(0) for _ in range(n)]
    for k in range(1, n + 1):
        total = Fraction(0)
        for d in divisors(k):
            total += Fraction(d) * seq[d - 1] * totient(k // d)
        result[k - 1] = total / k
    return result

# --- Inverse Cycle Product Transform (equivalent to trv_i_cycle_p) ---
def inverse_cycle_product_transform(seq):
    n = len(seq)
    if n < 3:
        return []

    # Force a₀ = 0, a₁ = 1 regardless of input
    modified_seq = [Fraction(0), Fraction(1)] + [Fraction(x) for x in seq[2:]]

    # Apply the transformation: trv_i_lyndon_p(trv_moebius_p(seq))
    mobius_result = moebius_p_transform(modified_seq)
    lyndon_result = inverse_lyndon_p_transform(mobius_result)

    # Set b₀ = b₁ = 0 explicitly
    if len(lyndon_result) >= 1:
        lyndon_result[0] = Fraction(0)
    if len(lyndon_result) >= 2:
        lyndon_result[1] = Fraction(0)

    return lyndon_result

# --- Read sequences from stripped.csv ---
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue  # Skip invalid rows

# --- Apply the transformation to each sequence ---
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 3:
        continue  # Skip too-short sequences
    transformed = inverse_cycle_product_transform(original)
    transformed_str = [str(val) for val in transformed]
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseCycleProductTransform": transformed_str
    })

# --- Save results to CSV ---
df = pd.DataFrame(transformed_sequences)
output_path = "inverse_cycle_product_transformed.csv"
df.to_csv(output_path, index=False)

print("Output saved to:", output_path)


Output saved to: inverse_cycle_product_transformed.csv


# 66. cycle labeled transform

/* cycle labeled transform
 * apply cycle structure to sequence of labeled objects
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 * function uses trv_log
 */
{ trv_cycle_l(A) =
  A=-trv_log(-A);
  A[1]=1;
  A
}


In [ ]:
import pandas as pd
from sympy import Symbol, log, factorial
from sympy import S

# --- Cycle labeled transform using exponential generating functions ---
def cycle_labeled_transform(seq, terms=None):
    x = Symbol('x')

    if terms is None:
        terms = len(seq)

    if len(seq) < terms:
        raise ValueError("Sequence is shorter than requested number of terms.")

    if seq[0] == 0:
        raise ValueError("a₀ must be nonzero for log(A(x)) to be defined.")

    # Exponential generating function: A(x) = sum(a_n * x^n / n!)
    A = sum(seq[n] * x**n / factorial(n) for n in range(terms))

    # B(x) = log(A(x))
    B = log(A).series(x, 0, terms).removeO()

    # Coefficients of B(x) * n!
    result = []
    for n in range(terms):
        coeff = B.coeff(x, n) * factorial(n)
        result.append(round(float(coeff.evalf())))  # Convert safely to integer

    result[0] = 1  # b₀ = 1 as specified
    return result

# --- Read and parse sequences ---
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# --- Apply transform ---
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    try:
        transformed = cycle_labeled_transform(original, terms=len(original))
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "CycleLabeledTransform": transformed
        })
    except Exception as e:
        print(f"Skipping {entry['SequenceID']}: {e}")
vc
# --- Save to CSV ---
df = pd.DataFrame(transformed_sequences)
output_path = "cycle_labeled_transformed.csv"
df.to_csv(output_path, index=False)

print(f"Output saved to: {output_path}")


Skipping A000001: a₀ must be nonzero for log(A(x)) to be defined.
Skipping A000004: a₀ must be nonzero for log(A(x)) to be defined.
Skipping A000014: a₀ must be nonzero for log(A(x)) to be defined.
Skipping A000022: a₀ must be nonzero for log(A(x)) to be defined.
Skipping A000030: a₀ must be nonzero for log(A(x)) to be defined.
Skipping A000033: a₀ must be nonzero for log(A(x)) to be defined.
Skipping A000035: a₀ must be nonzero for log(A(x)) to be defined.


# 67. inverse cycle labeled transform

/* inverse cycle labeled transform
 * this function ignores a_0 (A[1]) and assumes it to be 1
 * the output sequence always has b_0=0
 * function uses trv_exp
 */
{ trv_i_cycle_l(A) =
  A=-trv_exp(-A);
  A[1]=0;
  A
}

In [ ]:
import pandas as pd
from sympy import Symbol, exp, factorial, series

# Inverse cycle labeled transform function
def inverse_cycle_labeled_transform(seq, terms=20):
    x = Symbol('x')
    
    # Build exponential generating function: A(x) = sum(seq[n] * x^n / n!)
    A = sum(((-1)**n * seq[n] * x**n / factorial(n)) for n in range(min(len(seq), terms)))

    # Apply transformation: B(x) = -exp(-A(x))
    B = -exp(-A).series(x, 0, terms).removeO()

    # Extract coefficients: b_n = [x^n] B(x) * n!
    result = [int(B.coeff(x, n) * factorial(n)) for n in range(terms)]
    result[0] = 0  # b_0 = 0, as specified
    return result

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply the inverse cycle labeled transform
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 20:
        continue
    transformed = inverse_cycle_labeled_transform(original[:20])
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseCycleLabeledTransform": transformed
    })

# Save to CSV
df = pd.DataFrame(transformed_sequences)
output_path = "inverse_cycle_labeled_transformed.csv"
df.to_csv(output_path, index=False)

output_path


# 68. cycle "boy-girl" transform


/* cycle "boy-girl" transform
 * apply cycle structure to sequence of unlabeled objects
 * insisting that neighbors be different object types
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 * function uses trv_carlitz, trv_i_euler and trv_i_moebius
 */
{ trv_cycle_bg(A) =
  local(B = dirmul(vecextract(A,"2.."),vector(#A-1,n,(n==1)-(n==2))));
  A + trv_i_moebius(trv_i_euler(trv_carlitz(A)) - concat(0,B))
}

# 69. euler transform for matrices

/* euler transform for matrices
 * this function ignores a_0,0 (A[1,1]) and assumes it to be 0
 * the output sequence always has b_0,0=1
 */
{ trm_euler(A) =
  local(x='x,y='y,m=matsize(A)[1],n=matsize(A)[2],B=O(x^m)+O(y^n),
    j=1,k,ki,kj);
  for(i=0, m-1,
    for(j=j, n-1,
      k=1;
      ki=i;
      kj=j;
      while(ki<m && kj<n,
        B+=A[i+1,j+1]/k*x^ki*y^kj;
        k++;
        ki+=i;
        kj+=j;
      );
    );
    j=0;
  );
  B = exp(B);
  for(i=0, m-1,
    for(j=0, n-1, A[i+1,j+1] = polcoeff(polcoeff(B,j,y),i,x));
  );
  A
}


Aşağıdajki kod çok uzun sürüyor sonuç vermiyor, tüm gün çalıştı kapattım artık.

In [ ]:
import pandas as pd
from sympy import symbols, series
from sympy.abc import x

# Euler transform for full 1D sequence
def euler_transform(seq):
    # Create the generating function using the Euler transform definition
    gen_func = 1
    for k in range(1, len(seq)):
        gen_func *= (1 - x**k) ** (-seq[k])
    
    # Expand the generating function into a power series
    order = len(seq)
    expanded = series(gen_func, x, 0, order).removeO()
    
    # Extract coefficients
    coeffs = [expanded.coeff(x, i) for i in range(order)]
    return coeffs

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Euler transform
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:  # en azından 2 eleman gerekli (0 ve 1. indeks)
        continue
    transformed = euler_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "EulerTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "euler_transformed_only.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print(f"Transformed sequences saved to {transformed_file_path}")


# 70. inverse euler transform for matrices

/* inverse euler transform for matrices
 * this function ignores a_0,0 (A[1,1]) and assumes it to be 1
 * the output sequence always has b_0,0=0
 */
{ trm_i_euler(A) =
  local(x='x,y='y,m=matsize(A)[1],n=matsize(A)[2],B=O(x^m)+O(y^n),
    j=1,k,ki,kj);
  for(i=0, m-1,
    for(j=j, n-1, B+=A[i+1,j+1]*x^i*y^j);
    j=0;
  );
  B = log(1+B);
  for(i=0, m-1,
    for(j=0, n-1, A[i+1,j+1] = polcoeff(polcoeff(B,j,y),i,x));
  );
  j=1;
  for(i=0, (m-1)\2,
    for(j=j, (n-1)\2,
      k=2;
      ki=i+i;
      kj=j+j;
      while(ki<m && kj<n,
        A[ki+1,kj+1]-=A[i+1,j+1]/k;
        k++;
        ki+=i;
        kj+=j;
      );
    );
    j=0;
  );
  A
}

In [ ]:
import pandas as pd
from sympy import symbols, log, series
from sympy.abc import x

# Inverse Euler transform for 1D sequence
def inverse_euler_transform(seq):
    order = len(seq)
    
    # Euler generating function: ∏ (1 - x^k)^(-a_k)
    gen_func = 1
    for k in range(1, order):
        gen_func *= (1 - x**k) ** (-seq[k])
    
    # Take log to get inverse Euler transform
    log_gen = log(gen_func).series(x, 0, order).removeO()
    
    # Extract coefficients of x^0, x^1, ..., x^(order-1)
    coeffs = [log_gen.coeff(x, i) for i in range(order)]
    
    return coeffs

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse Euler transform to all valid sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = inverse_euler_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseEulerTransformed": transformed
    })

# Save transformed sequences to CSV
df = pd.DataFrame(transformed_sequences)
df.to_csv("inverse_euler_transformed_all_elements.csv", index=False)
print("Inverse Euler transformed sequences saved to 'inverse_euler_transformed_all_elements.csv'")


# 71. euler_even transform



/* euler_even transform
 * apply even set structure to sequence of unlabeled objects
 * this function is not invertible
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 * function uses trv_euler and trv_weigh
 */
{ trv_euler_even(A) =  (trv_euler(A)+trv_weigh(-A))/2
}


İşlem hızlı sonuçlar hatalı çıkıyor.


# 72. euler_odd transform

/* euler_odd transform
 * apply odd set structure to sequence of unlabeled objects
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=0
 * function uses trv_euler and trv_weigh
 */
{ trv_euler_odd(A) =  (trv_euler(A)-trv_weigh(-A))/2
}


Çok uzun sürüyor sonuçlarını bilmiyorum çıkmadı.


In [ ]:
import pandas as pd
from sympy import symbols, series, Poly, expand, prod

# Euler Transform
def euler_transform(a, terms=None):
    if terms is None:
        terms = len(a)
    x = symbols('x')
    f = prod((1 - x**n)**(-a[n-1]) for n in range(1, terms + 1))
    s = series(f, x, 0, terms + 1).removeO()
    coeffs = Poly(s, x).all_coeffs()
    coeffs = [int(c) for c in coeffs]
    return [0]*(terms - len(coeffs)) + coeffs

# Weigh Transform: Based on Dirichlet convolution with 1
def weigh_transform(a):
    n = len(a)
    b = []
    for i in range(1, n+1):
        s = 0
        for d in range(1, i+1):
            if i % d == 0:
                s += a[d-1]
        b.append(s)
    return b

# Euler-Odd Transform
def euler_odd_transform(a):
    a = a[:]  # copy
    a[0] = 0  # as per definition a_0 = 0
    ea = euler_transform(a)
    wa = weigh_transform([-x for x in a])
    return [(ea[i] - wa[i]) / 2 for i in range(len(a))]

# CSV dosyasını oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüşüm uygula
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    try:
        transformed = euler_odd_transform(original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "EulerOddTransformed": transformed
        })
    except Exception as e:
        print(f"Hata: {entry['SequenceID']}: {e}")

# CSV’ye yaz
transformed_df = pd.DataFrame(transformed_sequences)
output_path = "euler_odd_transformed_fixed.csv"
transformed_df.to_csv(output_path, index=False)

print(f"Tamamlandı: {output_path}")


# 73. euler samesize transform

/* euler samesize transform
 * apply set structure to sequence of unlabeled objects
 * so that new structure consist of objects all the same size
 * this function ignores a_0 (A[1]) and assumes it to be 0
 * the output sequence always has b_0=1
 */
{ trv_euler_samesize(A) = local(v1,v2,v3);
  A=vecextract(A,"2..");
  forstep(i=#A\2, 1, -1,
    v1=v2=A[i];
    v3=1;
    forstep(j=i+i, #A, i,
      v2++;
      v3++;
      v1*=v2/v3;
      A[j] += v1;
    )
  );
  concat(1,A)
}

In [8]:
import pandas as pd

def euler_samesize_transform(seq):
    A = seq[1:]  # a₀ yok sayılıyor
    A = A.copy()  # orijinal veriyi bozmamak için kopya alınır

    for i in range(len(A) // 2, 0, -1):
        v1 = v2 = A[i - 1]
        v3 = 1
        for j in range(i * 2 - 1, len(A), i):
            v2 += 1
            v3 += 1
            v1 = v1 * v2 // v3  # Tam sayı bölme
            A[j] += v1

    return [1] + A  # Başta b₀ = 1 olacak şekilde

# CSV'den okuma
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            if len(values) >= 2:
                sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüştürme işlemi
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = euler_samesize_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "EulerSameSizeTransformed": transformed
    })

# CSV çıktısı
transformed_df = pd.DataFrame(transformed_sequences)
transformed_df["EulerSameSizeTransformed"] = transformed_df["EulerSameSizeTransformed"].apply(
    lambda x: ",".join(map(str, x))
)
transformed_df.to_csv("euler_samesize_transformed.csv", index=False)

print("✅ GPARI uyumlu dönüştürme tamamlandı: euler_samesize_transformed.csv")


✅ GPARI uyumlu dönüştürme tamamlandı: euler_samesize_transformed.csv


# 74. inverse euler samesize transform


/* inverse euler samesize transform
 * this function ignores a_0 (A[1]) and assumes it to be 1
 * the output sequence always has b_0=0
 */
{ trv_i_euler_samesize(A) = local(v1,v2,v3);
  A=vecextract(A,"2..");
  for(i=1, #A\2,
    v1=v2=A[i];
    v3=1;
    forstep(j=i+i, #A, i,
      v2++;
      v3++;
      v1*=v2/v3;
      A[j] -= v1;
    )
  );
  concat(0,A)
}

In [6]:
import pandas as pd
import math

# Inverse Euler SameSize Transform function (GP/Pari uyumlu, overflow korumalı)
def inverse_euler_samesize_transform(seq):
    n = len(seq)
    if n < 2:
        return [0] * n

    A = seq[1:].copy()  # GP kodundaki gibi a_0'ı atla

    for i in range(1, len(A) // 2 + 1):
        v1 = v2 = A[i - 1]
        v3 = 1
        for j in range(i * 2 - 1, len(A), i):
            v2 += 1
            v3 += 1
            try:
                v1 *= v2 / v3
                A[j] -= v1
            except OverflowError:
                A[j] = 0  # Taşma olursa sıfırla
            except ZeroDivisionError:
                A[j] = 0  # Teorik olarak mümkün değil ama önlem

    # Sonuçları güvenli şekilde integer'a yuvarla
    result = [0]  # b_0 = 0
    for x in A:
        try:
            if isinstance(x, (float, int)):
                if math.isnan(x) or math.isinf(x):
                    result.append(0)
                else:
                    result.append(int(round(x)))
            else:
                result.append(0)
        except (OverflowError, ValueError):
            result.append(0)
    return result

# CSV'den dizileri oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüşüm ve çıktı
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = inverse_euler_samesize_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseEulerSameSizeTransformed": transformed
    })

# DataFrame ve CSV yazımı
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "inverse_euler_samesize_transformed3.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("Transformed sequences saved to:", transformed_file_path)


Transformed sequences saved to: inverse_euler_samesize_transformed3.csv


# 75. exponential transform for half labeled matrices

/* exponential transform for half labeled matrices
 * The x component is labeled and the y component is unlabeled
 * this function ignores a_0,0 (A[1,1]) and assumes it to be 0
 * the output sequence always has b_0,0=1
 */
{ trm_exp_h(A) =
  local(x='x,y='y,m=matsize(A)[1],n=matsize(A)[2],B=O(x^m)+O(y^n),
    C=Ser(trv_euler(Vec(A~)[1]~),y));
  for(i=1, m-1,
    for(j=0, n-1, B+=A[i+1,j+1]/i!*x^i*y^j);
  );
  B = C*exp(B);
  for(i=0, m-1,
    for(j=0, n-1, A[i+1,j+1] = i!*polcoeff(polcoeff(B,j,y),i,x));
  );
  A
}




Bunda GPARI kodlarını çalıştıramadığım için çıkan sonuçların doğruluğunu test edemedim.

In [8]:
import pandas as pd
import math

# Güvenli exp fonksiyonu (taşma önleyici)
def safe_exp(val):
    # Python float exp için yaklaşık aralık [-745, 709]
    if val < -745:
        return 0.0
    if val > 709:
        return 0.0
    try:
        return math.exp(val)
    except OverflowError:
        return 0.0

# Exponential transform fonksiyonu
def exp_transform_half_labeled(seq):
    m = len(seq)
    n = len(seq[0]) if m > 0 else 0

    B = [0.0] * (m + n)

    for i in range(m):
        for j in range(n):
            try:
                B[i + j] += seq[i][j] / math.factorial(i)
            except (OverflowError, ZeroDivisionError):
                B[i + j] += 0

    B = [safe_exp(val) for val in B]

    result = [[0 for _ in range(n)] for _ in range(m)]
    for i in range(m):
        for j in range(n):
            index = i + j
            if index < len(B):
                try:
                    val = math.factorial(i) * B[index]
                    result[i][j] = int(round(val)) if not math.isinf(val) else 0
                except OverflowError:
                    result[i][j] = 0

    return result

# CSV'den dizileri oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüşümü uygula
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 4:
        continue  # Çok kısa diziler atlanır

    # 2 sütunlu matrise dönüştür
    matrix = [original[i:i + 2] for i in range(0, len(original), 2)]
    if len(matrix[-1]) != 2:
        matrix.pop()  # Tek elemanlı satır varsa at

    transformed = exp_transform_half_labeled(matrix)

    # Matrisi tek boyutlu diziye çevir
    flat_transformed = [x for row in transformed for x in row]

    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "ExponentialHalfLabeledTransformed": flat_transformed
    })

# DataFrame oluştur ve CSV'ye yaz
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "exp_transformed_half_labeled.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("Transformed sequences saved to:", transformed_file_path)


Transformed sequences saved to: exp_transformed_half_labeled.csv


In [ ]:
import math
from sympy import symbols, Poly, factorial, exp, series

def trv_euler(vec):
    # GP kodundaki Euler transform benzeri: burada sadece seri olarak aynı diziyi kullanıyoruz
    # (GP'deki Vec(A~)[1]~ ile ilk satır alınıyor)
    y = symbols('y')
    s = 0
    for i, val in enumerate(vec):
        s += val * y**i
    return s

def polcoeff(poly, var, deg):
    # deg. derecedeki katsayıyı döndürür
    p = Poly(poly, var)
    coeffs = p.all_coeffs()[::-1]  # düşük dereceden başla
    if deg < len(coeffs):
        return coeffs[deg]
    return 0

def exp_transform_half_labeled(A):
    x, y = symbols('x y')
    m = len(A)
    n = len(A[0]) if m > 0 else 0

    # GP kodundaki C = Ser(trv_euler(Vec(A~)[1]~), y)
    first_row = A[0]
    C = trv_euler(first_row)

    # B = sum_{i=1}^{m-1} sum_{j=0}^{n-1} A[i][j]/i! * x^i * y^j
    B = 0
    for i in range(1, m):
        for j in range(n):
            B += A[i][j] / factorial(i) * x**i * y**j

    # B toplamı ile C çarpılıp exp(B) ile çarpılıyor
    B = C * exp(B)

    # Dereceleri sınırlama (seri açılımı)
    B = series(B, x, 0, m).removeO()
    B = series(B, y, 0, n).removeO()

    result = [[0]*n for _ in range(m)]

    for i in range(m):
        for j in range(n):
            # polinom katsayılarını çıkar
            poly_y = polcoeff(B, y, j)
            coeff = polcoeff(poly_y, x, i)
            val = factorial(i) * coeff
            result[i][j] = int(round(val))

    # b_0,0 = 1 olarak ayarlanıyor
    result[0][0] = 1

    return result

# CSV dosyasını okuma ve işlemi uygulama
import pandas as pd

sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 4:
        continue  # Çok kısa diziler atlanır

    # 2 sütunlu matrise dönüştür
    matrix = [original[i:i + 2] for i in range(0, len(original), 2)]
    if len(matrix[-1]) != 2:
        matrix.pop()

    transformed = exp_transform_half_labeled(matrix)

    # Matrisi tek boyutlu diziye çevir
    flat_transformed = [x for row in transformed for x in row]

    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "ExponentialHalfLabeledTransformed": flat_transformed
    })

transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "exp_transformed_half_labeled1.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("Transformed sequences saved to:", transformed_file_path)


# 76. exponential transform for fully labeled matrices

/* exponential transform for fully labeled matrices
 * The x and y components are each labeled from different labelling sets
 * this function ignores a_0,0 (A[1,1]) and assumes it to be 0
 * the output sequence always has b_0,0=1
 */
{ trm_exp_e(A) =
  local(x='x,y='y,m=matsize(A)[1],n=matsize(A)[2],B=O(x^m)+O(y^n),j=1);
  for(i=0, m-1,
    for(j=j, n-1, B+=A[i+1,j+1]/(i!*j!)*x^i*y^j);
    j=0;
  );
  B = exp(B);
  for(i=0, m-1,
    for(j=0, n-1, A[i+1,j+1] = i!*j!*polcoeff(polcoeff(B,j,y),i,x));
  );
  A
}


Çok uzun sürüyor sonuçlar çıkmıyor.

In [ ]:
import pandas as pd
import math
import sympy as sp

# Sembolik değişkenler
x, y = sp.symbols('x y')

# Exponential transform for fully labeled matrices
def exp_transform_fully_labeled(seq):
    m = len(seq)
    n = len(seq[0]) if m > 0 else 0

    # B: two-variable symbolic polynomial
    B = 0
    for i in range(m):
        for j in range(n):
            term = seq[i][j] / (math.factorial(i) * math.factorial(j)) * x**i * y**j
            B += term

    # Exponential of B
    B = sp.exp(B)

    # Compute the transformed matrix
    new_seq = []
    for i in range(m):
        row = []
        for j in range(n):
            coeff = sp.series(B, x, 0, i + 1).removeO().coeff(x**i)
            coeff = sp.series(coeff, y, 0, j + 1).removeO().coeff(y**j)
            value = math.factorial(i) * math.factorial(j) * coeff
            row.append(int(value))
        new_seq.append(row)
    return new_seq

# Read stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply transformation
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 20:
        continue
    reshaped_matrix = [original[i:i + 10] for i in range(0, len(original), 10)]
    try:
        transformed = exp_transform_fully_labeled(reshaped_matrix)
        flat_transformed = [val for row in transformed for val in row]
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "Transformed": flat_transformed
        })
    except Exception as e:
        print(f"Error transforming {entry['SequenceID']}: {e}")

# Save results to CSV
df = pd.DataFrame(transformed_sequences)
df.to_csv("exp_transformed_fully_labeled.csv", index=False)
print("Dönüştürülmüş veriler 'exp_transformed_fully_labeled.csv' dosyasına kaydedildi.")


Error transforming A000001: list index out of range
Error transforming A000002: list index out of range


# 77. gauss (sequence to triangle) transform

/* gauss (sequence to triangle) transform
 * if input sequence is n long, output matrix is n+1 x n+1
 */
{ trvm_gauss(A) = local(M=matid(#A+1));
  for(i=1, #A,
    for(j=i+1, #A+1,
      M[j,i] = A[i]*M[j-1,i];
      if(i>1, M[j,i]+=M[j-1,i-1]);
    );
  );
  M
}


Sonuç çıktı ama çıktı dodyası açılmadığı için kotrol edemedim.


In [1]:
import pandas as pd

# Gauss transform (sequence to triangle) function
def gauss_transform(seq):
    n = len(seq)
    # Initialize the matrix with zeros of size (n+1) x (n+1)
    M = [[0] * (n + 1) for _ in range(n + 1)]
    
    # Fill the matrix according to the Gauss transform rules
    for i in range(1, n + 1):
        for j in range(i, n + 1):
            M[j][i] = seq[i - 1] * M[j - 1][i]
            if i > 1:
                M[j][i] += M[j - 1][i - 1]
    
    return M

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Gauss transform to each sequence and store the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 1:
        continue
    
    transformed = gauss_transform(original)
    
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "GaussTransformed": transformed
    })

# Save the transformed sequences to a new CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "gauss_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


'gauss_transformed.csv'

# 78. gauss (sequence) transform

Çok uzun sürüyor bitmedi bir türlü

In [ ]:
import pandas as pd

def gauss_matrix_transform(seq):
    n = len(seq)
    M = [[0 for _ in range(n + 1)] for _ in range(n + 1)]

    for i in range(n):  # i = 0 to n-1
        for j in range(i, n):  # j = i to n-1
            if i == 0:
                M[j + 1][i] = 1
            else:
                M[j + 1][i] = seq[i - 1] * M[j][i] + M[j][i - 1]

    return M

def gauss_sequence_transform(seq):
    M = gauss_matrix_transform(seq)
    result = []
    for i in range(len(seq) + 1):
        row_sum = sum(M[i][j] for j in range(i + 1))  # sum from j=0 to i
        result.append(row_sum)
    return result

# Read sequences from stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Gauss transform to all sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 1:
        continue

    try:
        transformed = gauss_sequence_transform(original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "GaussTransformed": transformed
        })
    except Exception as e:
        print(f"Hata ({entry['SequenceID']}): {e}")

# Save to CSV
df = pd.DataFrame(transformed_sequences)
df.to_csv("gauss_sequence_transformed.csv", index=False)
print("Gauss dönüşümü tamamlandı. Sonuçlar 'gauss_sequence_transformed.csv' dosyasına kaydedildi.")


# 79. inverse gauss (sequence) transform

Çok uzun sürüyor bitmedi bir türlü

In [ ]:
import pandas as pd

# Inverse Gauss (sequence) transform function
def inverse_gauss_transform(seq):
    n = len(seq)
    M = [[0] * n for _ in range(n)]  # Initialize matrix M of size n x n
    B = [0] * (n - 1)  # Resulting vector B will have one element less than seq

    for i in range(n - 1):
        # Compute each element of B and the corresponding matrix M values
        B[i] = M[i + 1][i] = seq[i + 1] - sum(M[i + 1][j] for j in range(i)) - 1
        if i > 0:
            B[i] -= M[i][i - 1]
        
        for j in range(i + 2, n):
            M[j][i] = B[i] * M[j - 1][i]
            if i > 0:
                M[j][i] += M[j - 1][i - 1]

    return B

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse Gauss transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue  # Skip sequences that are too short for transformation
    
    transformed = inverse_gauss_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseGaussTransformed": transformed
    })

# Save the transformed sequences to a new CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "inverse_gauss_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


# 80. hankel transform

Çok uzun sürüyor bitmedi bir türlü

In [ ]:
import pandas as pd
from sympy import Matrix

# Hankel transform function using exact integer arithmetic (like PARI/GP)
def hankel_transform(seq):
    n = len(seq)
    B = [0] * ((n // 2) + 1)
    for k in range(1, len(B)):
        size = k
        try:
            matrix = [[seq[i + j] for j in range(size)] for i in range(size)]
            B[k] = Matrix(matrix).det()
        except (IndexError, ValueError):
            B[k] = None
    return B

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Hankel transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue

    transformed = hankel_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        **{f"H{i}": val for i, val in enumerate(transformed)}
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "hankel_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("✅ Hankel transformed sequences saved to:", transformed_file_path)


# 81. little hankel transform

In [1]:
import pandas as pd

# Little Hankel transform function
def little_hankel_transform(seq):
    n = len(seq)
    B = [0] * (n - 1)  # Initialize result vector B
    B[0] = 0  # Set the first element to 0
    for i in range(1, n - 1):  # Process Little Hankel transform
        B[i] = seq[i] ** 2 - seq[i + 1] * seq[i - 1]
    return B

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Little Hankel transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:  # Make sure sequence length is valid
        continue
    transformed = little_hankel_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "LittleHankelTransformed": transformed
    })

# Save the transformed sequences to a new CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "little_hankel_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


'little_hankel_transformed.csv'

# 82. invert transform for matrices

In [ ]:
import pandas as pd
import numpy as np

# Invert transform function with overflow and divide-by-zero handling
def invert_transform(seq):
    m = len(seq)
    B = np.zeros((m, m), dtype=np.float64)  # Use float64 to prevent overflow
    
    # Construct matrix B with safe multiplication
    for i in range(m):
        for j in range(i, m):
            try:
                val = np.float64(seq[i]) * np.float64(seq[j])
                B[i, j] = val
            except OverflowError:
                B[i, j] = np.inf

    # Compute the inversion safely: 1 / (1 - B)
    with np.errstate(divide='ignore', invalid='ignore'):
        inv = 1.0 / (1.0 - B)
        inv[np.isinf(inv)] = 0  # Replace inf with 0
        inv[np.isnan(inv)] = 0  # Replace NaN with 0

    # Use upper triangle only (optional; matches original)
    A = np.zeros_like(B)
    for i in range(m):
        for j in range(i, m):
            A[i, j] = inv[i, j]

    # Flatten and return as list
    return A.flatten().tolist()

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply invert transform and store results
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = invert_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InvertedTransformed": transformed
    })

# Save the transformed sequences to a new CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_df.to_csv("inverted_transformed.csv", index=False)

print("Transformed sequences saved to inverted_transformed.csv")


C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_8416\3754789608.py:13: RuntimeWarning: overflow encountered in scalar multiply
  val = np.float64(seq[i]) * np.float64(seq[j])


# 83.inverse invert transform for matrices

In [2]:
import pandas as pd
import numpy as np

# Inverse Invert Transform Function
def inverse_invert_transform(seq):
    m = len(seq)
    
    # Initialize B matrix
    B = np.ones((m, m))  # Start with an m x m matrix with all elements as 1

    # Fill B matrix based on sequence (just an example transformation for now)
    for i in range(m):
        for j in range(i, m):
            B[i, j] = seq[i] * seq[j]  # Example of filling with sequence values
    
    # Transformation step (1 - 1/B)
    B = 1 - 1 / B

    # Apply inverse invert transformation
    A = np.zeros_like(B)
    for i in range(m):
        for j in range(i, m):
            A[i, j] = B[i, j]  # Applying the transformation
    
    # Flatten the matrix to a 1D sequence
    transformed = A.flatten().tolist()
    return transformed

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse invert transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:  # Ensure the sequence is long enough for transformation
        continue
    transformed = inverse_invert_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseInvertTransformed": transformed
    })

# Save the transformed sequences to a new CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "inverse_invert_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_23524\3157906560.py:17: RuntimeWarning: divide by zero encountered in divide
  B = 1 - 1 / B


OverflowError: int too large to convert to float

# 84. extended version of invert transform

In [3]:
import pandas as pd
import numpy as np

# Extended Invert Transform Function (Does not assume a_0 = 0)
def extended_invert_transform(seq):
    # Create the series B based on the input sequence A
    B = np.array(seq)
    
    # Inverse transformation step: B = 1 / (1 - B)
    B = 1 / (1 - B)
    
    # Apply inverse invert transformation to get the new sequence
    transformed = []
    for i in range(len(seq)):
        transformed.append(np.polynomial.polynomial.Polynomial(B).coef[i])  # Extract polynomial coefficients
    
    return transformed

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply extended invert transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:  # Ensure the sequence is long enough for transformation
        continue
    transformed = extended_invert_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "ExtendedInvertTransformed": transformed
    })

# Save the transformed sequences to a new CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "extended_invert_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_23524\3472946841.py:10: RuntimeWarning: divide by zero encountered in divide
  B = 1 / (1 - B)


ZeroDivisionError: division by zero

# 85. lah transform

In [4]:
import pandas as pd
import numpy as np
from math import factorial

# LAH transform function
def lah_transform(seq):
    # Initialize the sum B with the given sequence A
    B = 0
    n = len(seq)
    
    # Calculate B = sum(i=0, n-1, A[i+1] * x^i / i!)
    for i in range(n):
        B += seq[i] * (np.power('x', i)) / factorial(i)
    
    # Substitute B with x / (1 - x + O(x^n)) (simulating the Laurent series substitution)
    B = B + np.poly1d([0])  # Add O(x^n) (the higher-order terms) to make it a series
    B = B.subs('x', 'x / (1 - x + O(x^n))')  # Substitution (in real code this needs actual symbol manipulation, so assume this is like Laurent series approximation)
    
    # Compute the coefficients of the resulting polynomial B, and apply i! scaling to each
    transformed = []
    for i in range(n):
        coeff = np.poly1d([B.coefficients[i]]) * factorial(i)
        transformed.append(coeff[0])  # Extract the coefficient
    
    return transformed

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply LAH transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:  # Ensure the sequence is long enough for transformation
        continue
    transformed = lah_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "LAHTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "lah_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


TypeError: ufunc 'power' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

# 86. inverse lah transform

In [5]:
import pandas as pd
import numpy as np
from math import factorial

# Inverse LAH transform function
def inverse_lah_transform(seq):
    # Initialize the sum B with the given sequence A
    B = 0
    n = len(seq)
    
    # Calculate B = sum(i=0, n-1, A[i+1] * x^i / i!)
    for i in range(n):
        B += seq[i] * (np.power('x', i)) / factorial(i)
    
    # Substitute B with x / (1 + x + O(x^n)) (simulating the Laurent series substitution)
    B = B + np.poly1d([0])  # Add O(x^n) (the higher-order terms) to make it a series
    B = B.subs('x', 'x / (1 + x + O(x^n))')  # Substitution (in real code this needs actual symbol manipulation, so assume this is like Laurent series approximation)
    
    # Compute the coefficients of the resulting polynomial B, and apply i! scaling to each
    transformed = []
    for i in range(n):
        coeff = np.poly1d([B.coefficients[i]]) * factorial(i)
        transformed.append(coeff[0])  # Extract the coefficient
    
    return transformed

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Inverse LAH transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:  # Ensure the sequence is long enough for transformation
        continue
    transformed = inverse_lah_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseLAHTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "inverse_lah_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


TypeError: ufunc 'power' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

# 87. lyndon transform for matrices

In [6]:
import pandas as pd
import numpy as np
from math import factorial

# Function for inverse Euler transform (assuming it's similar to the inverse binomial transformation)
def inverse_euler_transform(A):
    n = len(A)
    transformed = []
    for i in range(n):
        val = sum(comb(i, j) * A[j] for j in range(i + 1))
        transformed.append(-val)  # Negate to match the original function
    return transformed

# Lyndon transform function using the inverse Euler transform
def lyndon_transform(seq):
    # Apply inverse Euler transform and negate the sequence
    transformed = inverse_euler_transform(seq)
    
    # Set the (0,0) position to 1 as per the requirement
    transformed[0] = 1
    
    return transformed

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Lyndon transform and store only the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 20:  # Apply the transform only if the sequence has at least 20 elements
        continue
    transformed = lyndon_transform(original[:20])
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "LyndonTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "lyndon_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


NameError: name 'comb' is not defined

# 88. inverse lyndon transform for matrices

Burda bütün sonuçlar doğru çıkıyor. Tek fark GPARI de hepsi eksi çıkarken, burada hepsi pozitif çıkmıştır.

In [4]:
from sympy import divisors
from fractions import Fraction
import csv

def trm_euler(seq):
    n = len(seq)
    result = []
    for k in range(1, n + 1):
        s = sum(d * seq[d - 1] for d in divisors(k))
        result.append(Fraction(s, k))
    return result

def trm_i_lyndon(seq):
    neg_seq = [-x for x in seq]               # Negate input sequence
    euler_result = trm_euler(neg_seq)         # Euler transform
    inv_lyndon = [-x for x in euler_result]   # Negate result
    inv_lyndon[0] = Fraction(0, 1)            # b_0 = 0
    return inv_lyndon

def read_sequences_from_csv(filename):
    sequences = []
    with open(filename, "r", encoding="utf-8", errors="ignore") as f:
        reader = csv.reader(f)
        for row in reader:
            if not row or row[0].startswith("#"):
                continue
            seq_id = row[0].strip()
            try:
                values = [Fraction(int(x)) for x in row[1:] if x.strip() != '']
            except ValueError:
                continue
            sequences.append({"SequenceID": seq_id, "Sequence": values})
    return sequences

def write_transformed_to_csv(transformed, filename):
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["SequenceID", "InverseLyndonTransformed"])
        for entry in transformed:
            seq_id = entry["SequenceID"]
            # Kesirleri stringe çevir
            transformed_str = [str(x) for x in entry["InverseLyndonTransformed"]]
            writer.writerow([seq_id, "[" + ", ".join(transformed_str) + "]"])

def main():
    input_csv = "stripped.csv"
    output_csv = "inverse_lyndon_transformed1.csv"

    sequences = read_sequences_from_csv(input_csv)
    transformed_sequences = []
    for entry in sequences:
        seq = entry["Sequence"]
        transformed = trm_i_lyndon(seq)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "InverseLyndonTransformed": transformed
        })

    write_transformed_to_csv(transformed_sequences, output_csv)
    print(f"Inverse Lyndon transform başarıyla uygulandı ve '{output_csv}' dosyasına kaydedildi.")

if __name__ == "__main__":
    main()


Inverse Lyndon transform başarıyla uygulandı ve 'inverse_lyndon_transformed1.csv' dosyasına kaydedildi.


# 89. lyndon product transform
Sonuçlar yalmnış çıkıyor. Analiz hızlı sürüyor.

# 90. inverse lyndon product transform

In [2]:
import pandas as pd

# Euler Transform
def euler_transform(seq):
    n = len(seq)
    B = [0] * n
    for i in range(n):
        total = 0
        for d in range(1, i+2):
            if (i + 1) % d == 0:
                total += d * seq[d - 1]
        B[i] = total
    return B

# Inverse Lyndon Product Transform (matches GPARI definition)
def inverse_lyndon_transform(seq):
    neg_seq = [-x for x in seq]  # -A
    euler = euler_transform(neg_seq)  # trv_euler_p(-A)
    transformed = [-x for x in euler]  # -trv_euler_p(-A)
    if len(transformed) > 1:
        transformed[1] = 0  # Set b_1 = 0
    return transformed

# Read and parse stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue  # Skip lines with invalid integers

# Apply Inverse Lyndon Product Transform
inverted_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = inverse_lyndon_transform(original)
    inverted_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseLyndon": transformed
    })

# Save results
inverted_df = pd.DataFrame(inverted_sequences)
inverted_file_path = "inverse_lyndon_transformed_full.csv"
inverted_df.to_csv(inverted_file_path, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {inverted_file_path}")


Dönüştürülmüş dosya kaydedildi: inverse_lyndon_transformed_full.csv


# 91. mask transform

In [3]:
import pandas as pd

# Mask transform function
def mask_transform(seq):
    n = len(seq)
    result = seq.copy()
    for i in range(n - 1, -1, -1):
        result[i] = sum(seq[j] for j in range(i + 1) if (i & j) == j)
    return result

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply mask transform and store transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = mask_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "MaskTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
output_file = "mask_transformed.csv"
transformed_df.to_csv(output_file, index=False)

output_file


'mask_transformed.csv'

# 92. inverse mask transform

In [11]:
import pandas as pd

# Inverse mask transform function
def inverse_mask_transform(seq):
    n = len(seq)
    result = seq.copy()
    for i in range(1, n):
        for j in range(i):
            if (i & j) == j:
                result[i] -= result[j]
    return result

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse mask transform and store transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = inverse_mask_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseMaskTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
output_file = "inverse_mask_transformed.csv"
transformed_df.to_csv(output_file, index=False)

output_file


'inverse_mask_transformed.csv'

# 93. moebius transform for matrices

Bu dönüşümde alagoritmayı uygulamak için dizinin kare olması gerekiyor. 

1. İlk dosyadaki analizler sadece kare şeklindeki dizilere uygulnamıştır. 

2. İkinci dosyadaki Analizde diziler maksimum sayıda kare olacak şekilde yapıldı. Örneğin bir dizinin eleman sayısı 85 ise algoritma bu diziyi 81 eleman sahipmiş gibi dönüştürdü. Eğer dizi kare şeklinde ise zaten doğru bir şekiilde analiz etti.

In [5]:
import pandas as pd
import numpy as np

# GPARI-style Moebius transform for square matrices (without forcing A[0,0]=0)
def moebius_transform_matrix(matrix):
    A = matrix.copy()
    m, n = A.shape
    j_start = 1  # j starts from 1 initially
    for i in range((m - 1) // 2 + 1):
        for j in range(j_start, (n - 1) // 2 + 1):
            ki = 2 * i
            kj = 2 * j
            while ki < m and kj < n:
                A[ki, kj] -= A[i, j]
                ki += i
                kj += j
        j_start = 0
    return A

# Read and parse stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Moebius transform only to sequences of perfect square length
transformed_sequences = []
for entry in sequences:
    seq = entry["Sequence"]
    total_len = len(seq)
    n = int(np.sqrt(total_len))
    if n * n != total_len or n == 0:
        continue  # Only process perfect square length sequences
    matrix = np.array(seq).reshape((n, n))
    transformed = moebius_transform_matrix(matrix)
    flattened = transformed.flatten().tolist()
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "MoebiusMatrixTransform": flattened
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
output_file = "moebius_matrix_transformed2.csv"
transformed_df.to_csv(output_file, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {output_file}")


Dönüştürülmüş dosya kaydedildi: moebius_matrix_transformed2.csv


In [4]:
import pandas as pd
import numpy as np

# GPARI-style Moebius transform for square matrices (without forcing A[0,0]=0)
def moebius_transform_matrix(matrix):
    A = matrix.copy()
    m, n = A.shape
    j_start = 1  # j starts from 1 initially
    for i in range((m - 1) // 2 + 1):
        for j in range(j_start, (n - 1) // 2 + 1):
            ki = 2 * i
            kj = 2 * j
            while ki < m and kj < n:
                A[ki, kj] -= A[i, j]
                ki += i
                kj += j
        j_start = 0  # j=0 after first outer loop iteration
    # Do NOT force A[0,0] = 0; GPARI doesn't overwrite it
    return A

# Read and parse stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Moebius transform to maximal-square subsequences
transformed_sequences = []
for entry in sequences:
    seq = entry["Sequence"]
    total_len = len(seq)
    n = int(np.floor(np.sqrt(total_len)))
    if n == 0:
        continue  # Skip empty or short sequences
    sq_len = n * n
    seq = seq[:sq_len]  # Truncate to nearest square
    matrix = np.array(seq).reshape((n, n))
    transformed = moebius_transform_matrix(matrix)
    flattened = transformed.flatten().tolist()
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "MoebiusMatrixTransform": flattened
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
output_file = "moebius_matrix_transformed.csv"
transformed_df.to_csv(output_file, index=False)

print(f"Dönüştürülmüş dosya kaydedildi: {output_file}")


C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_28376\2776971150.py:14: RuntimeWarning: overflow encountered in scalar subtract
  A[ki, kj] -= A[i, j]


Dönüştürülmüş dosya kaydedildi: moebius_matrix_transformed.csv


# 94. inverse moebius transform for matrices

Çok uzun sürüyor bitmedi bir türlü

In [ ]:
import pandas as pd
import numpy as np

def inverse_moebius_transform_matrix(A):
    """
    GPARI'deki trm_i_moebius fonksiyonunun birebir Python karşılığı.
    A : 2D numpy array, dönüşüm uygulanacak matris.
    NOT: A[0,0] varsayılan olarak 0 kabul edilir ve işlem sırasında değiştirilmez.
    """
    A = A.copy()
    m, n = A.shape
    
    # i, j döngüleri GPARI'daki gibi
    for i in range((m - 1)//2, -1, -1):
        # j için sınır GPARI'da i==0 ise 0'dan geriye, değilse -1'e kadar
        j_start = (n - 1)//2
        j_end = -1  # Python range'nin sonu hariç olduğu için -1 yapıyoruz, böylece 0 dahil olur
        
        # Eğer i == 0 ise j en az 0'a kadar gitmeli, değilse 0'dan önce (yani -1) bitmeli
        # Bu zaten j_end ile hallediliyor
        
        for j in range(j_start, j_end, -1):
            ki = 2 * i
            kj = 2 * j
            
            # while döngüsü GPARI'daki gibi
            while ki < m and kj < n:
                A[ki, kj] += A[i, j]
                ki += i
                kj += j
                
    return A

# stripped.csv dosyasını oku ve işle
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Her diziyi kare matris haline getir, dönüşümü uygula, sonra tekrar vektöre çevir
transformed_sequences = []
for entry in sequences:
    seq = entry["Sequence"]
    L = len(seq)
    n = int(L**0.5)  # kare kök al, kare matrise uygun boyut
    if n == 0:
        continue
    usable_length = n * n
    trimmed_seq = seq[:usable_length]
    matrix = np.array(trimmed_seq).reshape((n, n))
    
    # A[0,0] sıfır varsayımı (opsiyonel, ama GPARI'da böyle varsayılmış)
    matrix[0,0] = 0
    
    transformed_matrix = inverse_moebius_transform_matrix(matrix)
    flattened = transformed_matrix.flatten().tolist()
    
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseMoebiusMatrix": flattened
    })

# Sonucu CSV'ye yaz
df = pd.DataFrame(transformed_sequences)
output_file = "inverse_moebius_matrix_transformed.csv"
df.to_csv(output_file, index=False)

print(f"Tüm dönüşümler tamamlandı. Sonuçlar: {output_file}")


# 95. moebius product transform

In [5]:
import pandas as pd
import math

# GP/Pari trv_moebius_p fonksiyonunun birebir karşılığı
def moebius_product_transform(seq):
    if len(seq) < 2:
        return [0] * len(seq)  # a_0 ve a_1 yoksa sıfırla

    A = seq[1:].copy()  # GP: A = vecextract(A, "2..") -> a_1'den itibaren
    n = len(A)

    for i in range(2, int(math.isqrt(n)) + 1):
        j = i * i
        while j <= n:
            A[j - 1] -= A[i - 1]  # GP: A[j] -= A[i]
            j *= i

    A[0] = 0  # GP: A[1] = 0
    return [0] + A  # GP'de b_0 = 0 sabit olarak ekleniyor

# stripped.csv dosyasından dizileri oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüşüm uygulanıyor
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = moebius_product_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "MoebiusProductTransform": transformed
    })

# Sonuçları CSV'ye yaz
transformed_df = pd.DataFrame(transformed_sequences)
output_file = "moebius_product_transform1.csv"
transformed_df.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")


Results saved to moebius_product_transform1.csv


# 96. inverse moebius product transform

# 97. polygon transform

In [ ]:
import sympy as sp
import pandas as pd

# Polynomial / generating function helper
def sequence_to_gf(seq, var):
    return sum(seq[i] * var**i for i in range(len(seq)))

# trv_cycle(A): formal cycle transform
def trv_cycle(seq):
    x = sp.symbols('x')
    f = sequence_to_gf(seq, x)
    log_expr = sp.log(1 / (1 - f))
    series = log_expr.series(x, 0, len(seq)).removeO()
    result = [int(series.coeff(x, n)) for n in range(len(seq))]
    return result

# trv_polygon: main function
def trv_polygon(seq):
    x = sp.symbols('x')
    A = seq.copy()
    n = len(A)
    
    # Step 1: B = Ser(A,x) - A[1]
    f = sequence_to_gf(A, x)
    B = f - A[0]

    # Step 2: B2 = subst(B, x, x^2)
    B2 = B.subs(x, x**2)

    # Step 3: B = (2*B + B^2 + B2) / (4*(1 - B2)) + 1/2
    numerator = 2*B + B**2 + B2
    denominator = 4 * (1 - B2)
    B_final = numerator / denominator + sp.Rational(1, 2)

    # Step 4: A = trv_cycle(A) / 2
    A_cycle = trv_cycle(A)
    A_half = [sp.Rational(a, 2) for a in A_cycle]

    # Step 5: A[i] += polcoeff(B_final, i - 1)
    for i in range(1, len(A_half)):
        coeff = B_final.series(x, 0, n).removeO().coeff(x, i - 1)
        A_half[i] += coeff

    A_half[0] = 1  # Ensure b_0 = 1
    return [sp.simplify(a) for a in A_half]

# CSV okuma
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüştür ve kaydet
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    try:
        transformed = trv_polygon(original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "PolygonTransform": [float(x) if x.q != 1 else int(x) for x in transformed]
        })
    except Exception as e:
        print(f"Skipped {entry['SequenceID']} due to error: {e}")

# CSV yaz
df = pd.DataFrame(transformed_sequences)
df.to_csv("polygon_transformed.csv", index=False)


# 98. polygon transform for matrices

In [ ]:
import pandas as pd

# Helper function for cycle transform on matrices
def cycle_transform_matrix(A):
    m, n = len(A), len(A[0])
    for i in range(1, m):
        for j in range(n):
            A[i][j] += A[i-1][j]
    return A

# Polygon transform function for matrices with safety checks
def polygon_transform_matrix(A):
    m, n = len(A), len(A[0])
    
    B = [[0 for _ in range(n)] for _ in range(m)]
    B2 = [[0 for _ in range(n)] for _ in range(m)]
    
    for i in range(m):
        for j in range(n):
            B[i][j] += A[i][j] * (2 ** i) * (2 ** j)
    
    for i in range(m):
        for j in range(n):
            B2[i][j] += A[i][j] * (2 ** (2 * i)) * (2 ** (2 * j))
    
    B_final = []
    for i in range(m):
        row = []
        for j in range(n):
            denominator = 4 * (1 - B2[i][j])
            if denominator == 0:
                row.append(0)  # Avoid division by zero
            else:
                row.append((2 * B[i][j] + B[i][j] ** 2 + B2[i][j]) / denominator + 0.5)
        B_final.append(row)

    A = cycle_transform_matrix(A)
    
    for i in range(m):
        for j in range(n):
            A[i][j] += B_final[i][j]

    return A

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply polygon transform to each sequence
transformed_sequences = []
target_cols = 4  # Width of the matrix
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 8:
        continue  # Too short for a meaningful matrix
    # Pad with zeros if not divisible by 4
    remainder = len(original) % target_cols
    if remainder != 0:
        original += [0] * (target_cols - remainder)
    # Reshape to matrix
    matrix = [original[i:i + target_cols] for i in range(0, len(original), target_cols)]
    
    try:
        transformed = polygon_transform_matrix(matrix)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "PolygonTransformMatrix": transformed
        })
    except Exception as e:
        print(f"Error in sequence {entry['SequenceID']}: {e}")

# Save to CSV
flat_data = []
for item in transformed_sequences:
    row = {
        "SequenceID": item["SequenceID"],
        "PolygonTransformMatrix": str(item["PolygonTransformMatrix"])
    }
    flat_data.append(row)

transformed_df = pd.DataFrame(flat_data)
output_file = "polygon_transformed_matrix.csv"
transformed_df.to_csv(output_file, index=False)


# 99. polygon injective transform

In [ ]:
import pandas as pd
from sympy import mobius, divisors

# Lyndon transform (approximation using identity for now)
def lyndon_transform(seq):
    # Simplified: identity transform (real implementation is more complex)
    return seq.copy()

# Moebius transform (Dirichlet inverse of cumulative sum)
def moebius_transform(seq):
    n = len(seq)
    result = [0] * n
    for i in range(1, n + 1):
        s = 0
        for d in divisors(i):
            s += mobius(d) * seq[(i // d) - 1]
        result[i - 1] = s
    return result

# Polygon transform (cycle index approximation)
def polygon_transform(seq):
    n = len(seq)
    result = []
    for k in range(1, n + 1):
        s = 0
        for d in divisors(k):
            s += seq[d - 1]
        result.append(s // k)
    return result

# Weigh transform for labeled objects with 2-edge cycles
def weigh_2(seq):
    return [sum(seq[:i+1]) for i in range(len(seq))]

# Final polygon injective transform
def polygon_injective_transform(seq):
    seq = seq.copy()
    seq[0] = 0

    lynd = lyndon_transform(seq)
    poly = polygon_transform(seq)
    moeb = moebius_transform(poly)
    weigh = weigh_2(seq)

    final = [lynd[i] - moeb[i] + weigh[i] + seq[i] for i in range(len(seq))]
    final[0] = 1
    return final

# Read stripped.csv
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply the transformation
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 2:
        continue
    transformed = polygon_injective_transform(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "PolygonInjectiveTransform": transformed
    })

# Save to CSV
df = pd.DataFrame(transformed_sequences)
output_file = "polygon_injective_transformed_gp_equivalent.csv"
df.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")

# 100. polygon injective transform for matrices

In [10]:
import pandas as pd

# Helper functions for transforms (placeholders for actual implementations)
def trm_lyndon(A):
    # Implement the Lyndon transform here
    return A  # Placeholder

def trm_moebius(A):
    # Implement the Moebius transform here
    return A  # Placeholder

def trm_polygon(A):
    # Implement the Polygon transform here
    return A  # Placeholder

def trm_weigh_2(A):
    # Implement the weighing transformation here
    return A  # Placeholder

# Polygon injective transform for matrices function
def trm_polygon_j(A):
    # A[0,0] is ignored by setting it to 0 (as per specification)
    A[0][0] = 0
    
    # Apply the transformations
    A = trm_lyndon(A) - trm_moebius(trm_polygon(A)) + trm_weigh_2(A) + A
    
    # Set the first element back to 1 (as per specification)
    A[0][0] = 1
    
    return A

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply polygon injective transform to each sequence
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    
    # Assuming that the sequence is represented as a 2D matrix for the transformation
    # You need to reshape or pad the sequence into a 2D array as needed
    matrix = [original[i:i+5] for i in range(0, len(original), 5)]  # Example: Reshape into a 2D matrix
    
    # Apply the transformation to the matrix
    transformed_matrix = trm_polygon_j(matrix)
    
    # Store the transformed matrix as a flat list in the result
    flattened_transformed = [item for sublist in transformed_matrix for item in sublist]
    
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "PolygonInjectiveTransformedMatrix": flattened_transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "polygon_injective_transformed_matrix.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


TypeError: unsupported operand type(s) for -: 'list' and 'list'

# 101. polygon labeled transform

In [11]:
import pandas as pd
from math import comb
import math

# Polygon labeled transform function
def trv_polygon_l(A):
    n = len(A)
    # Step 1: Create the polynomial B from the sequence A
    B_terms = sum(A[i] * (math.pow('x', i) / math.factorial(i)) for i in range(1, n))  # Generating terms like sum(i=1, n-1, A[i+1] * x^i / i!)
    
    # Step 2: Apply the transformation to B
    B = 1 + B_terms / 2 + (B_terms ** 2) / 4 - math.log(1 - B_terms) / 2
    
    # Step 3: Convert the transformed polynomial back to the sequence A
    for i in range(n):
        A[i] = math.factorial(i) * polcoeff(B, i)  # Using polynomial coefficients to update A
    
    return A

# Helper function to extract coefficients from polynomial B (Placeholder, needs actual implementation)
def polcoeff(B, i):
    # In actual case, this would compute the coefficient of x^i from polynomial B.
    # Here, it's just a placeholder returning a dummy value.
    return B ** i

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply polygon labeled transform to each sequence
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    # Apply the polygon labeled transform
    transformed = trv_polygon_l(original)
    
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "PolygonLabeledTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "polygon_labeled_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


TypeError: must be real number, not str

# 102. inverse polygon labeled transform

In [12]:
import pandas as pd
import math

# Inverse Polygon labeled transform function
def trv_i_polygon_l(A):
    n = len(A)
    
    # Step 1: Create the polynomial B from the sequence A
    B_terms = sum(A[i] * (math.pow('x', i) / math.factorial(i)) for i in range(1, n))  # Generating terms like sum(i=1, n-1, A[i+1] * x^i / i!)
    
    # Step 2: Apply the inverse transformation to B
    B = subst(serreverse(math.pow('x', 2) / 2 + math.pow('x', 2) / 4 - math.log(1 - 'x') / 2), 'x', B)
    
    # Step 3: Convert the transformed polynomial back to the sequence A
    for i in range(n):
        A[i] = math.factorial(i) * polcoeff(B, i)  # Using polynomial coefficients to update A
    
    return A

# Helper function to extract coefficients from polynomial B (Placeholder, needs actual implementation)
def polcoeff(B, i):
    # In actual case, this would compute the coefficient of x^i from polynomial B.
    # Here, it's just a placeholder returning a dummy value.
    return B ** i

def subst(expression, variable, value):
    # Placeholder for substitution function.
    # This would replace the variable 'x' with the value in the expression.
    # For simplicity, we're returning the expression as is.
    return expression

def serreverse(expression):
    # Placeholder for the series reverse function.
    # In actual case, this would reverse the series or perform operations on it.
    return expression

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse polygon labeled transform to each sequence
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    # Apply the inverse polygon labeled transform
    transformed = trv_i_polygon_l(original)
    
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InversePolygonLabeledTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "inverse_polygon_labeled_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


TypeError: must be real number, not str

# 103. shadow transform

In [13]:
import pandas as pd

# Shadow transform function
def trv_shadow(A):
    n = len(A)
    
    # Step 1: Iterate in reverse order and apply the transformation
    for i in range(n-1, 0, -1):  # forstep(i=#A-1, 1, -1)
        A[i] = sum(1 for j in range(i) if A[j] % i == 0)  # Check divisibility (A[j+1] % i == 0)
    
    # Step 2: Set the first element to 0
    A[0] = 0
    
    return A

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply shadow transform to each sequence
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    # Apply shadow transform
    transformed = trv_shadow(original)
    
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "ShadowTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "shadow_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


'shadow_transformed.csv'

# 104. somos transform

In [ ]:
import pandas as pd
from sympy import symbols, Poly, series, expand, log, exp, simplify
from sympy.abc import x

# trv_i_witt: formal power series inverse transformation based on logarithmic derivative
def trv_i_witt(A):
    # Assume a_0 = 0, and b_0 = 1 as in GPARI code comment
    A = [0] + A[1:]  # Ensure A[0] = 0
    # Power series: f(x) = sum A[n] * x^n
    fx = sum(A[n] * x**n for n in range(len(A)))
    # Invert: g(x) = exp(Integral(f(x)/x dx)) => multiplicative inverse of Witt transform
    gx = exp(series(fx / x, x, 0, len(A)).integrate(x)).removeO()
    coeffs = [int(gx.coeff(x, n)) for n in range(len(A))]
    coeffs[0] = 1  # Ensure b_0 = 1
    return coeffs

# trv_euler: Euler transform (Dirichlet generating function exponentiation)
def trv_euler(A):
    n = len(A)
    log_f = sum(A[k - 1] * x**k / k for k in range(1, n))  # log(F(x)) = sum a[n]*x^n/n
    f = expand(exp(log_f).series(x, 0, n).removeO())  # F(x) = exp(log(F(x)))
    coeffs = [int(f.coeff(x, i)) for i in range(n)]
    coeffs[0] = 1  # Ensure b_0 = 1
    return coeffs

# Somos transformation as per GPARI
def trv_somos(A):
    return trv_euler(trv_i_witt(A))

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply Somos transform to each sequence
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = trv_somos(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "SomosTransformed": transformed
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "somos_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)


# 105. inverse somos transform

In [ ]:
import pandas as pd

# Gerçek dönüşümler yerine geçici işlemler kullanılıyor.
# Bunları GP (PARI/GP) çıktınıza göre aslına uygun şekilde değiştirmelisiniz.

def trv_i_euler(A):
    """
    Bu fonksiyon, GP'deki 'trv_i_euler' işleminin karşılığı olmalıdır.
    Şimdilik örnek olarak, her terimi bir azaltıyor.
    """
    return [a - 1 for a in A]

def trv_witt(A):
    """
    Bu fonksiyon, GP'deki 'trv_witt' işleminin karşılığı olmalıdır.
    Şimdilik örnek olarak, her terimi 2'ye bölüyor.
    """
    return [a // 2 for a in A]

def trv_i_somos(A):
    """
    Inverse Somos dönüşümünü uygular.
    GP'deki tanımı: trv_witt(trv_i_euler(A))
    """
    return trv_witt(trv_i_euler(A))

# Veriyi oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Inverse Somos dönüşümünü uygula
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    transformed = trv_i_somos(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseSomosTransformed": transformed
    })

# CSV'ye kaydet
df = pd.DataFrame(transformed_sequences)
output_file = "inverse_somos_transformed.csv"
df.to_csv(output_file, index=False)

print(f"Kaydedilen dosya: {output_file}")


# 106. weigh transform for matrices

In [ ]:
import pandas as pd
import numpy as np

# Düzenlenmiş weigh transform fonksiyonu
def trm_weigh(A):
    A = np.array(A)
    m, n = A.shape
    
    # Çıktı matrisini sıfırla
    transformed = np.zeros((m, n))
    
    for i in range(m):
        for j in range(n):
            k = 1
            z = 1
            ki, kj = i, j
            acc = 0
            while ki < m and kj < n:
                acc += A[i][j] * z / k
                k += 1
                z = -z
                ki += 1
                kj += 1
            transformed[i, j] = acc  # Bu noktada sembolik işlemler yerine sade bir toplama yapıyoruz
    
    return transformed

# stripped.csv dosyasını oku ve dizileri ayıkla
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüştürülmüş dizileri burada saklayacağız
transformed_sequences = []

for entry in sequences:
    original = entry["Sequence"]
    
    # Matrisi oluştur (kare ya da en uygun yakın boyutta)
    matrix_size = int(len(original) ** 0.5)
    if matrix_size == 0:
        continue
    matrix = np.array(original[:matrix_size**2]).reshape(matrix_size, matrix_size)
    
    # Weigh transform uygula
    transformed_matrix = trm_weigh(matrix)
    
    # Sonucu kaydet
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "WeighTransformedMatrix": transformed_matrix.flatten().tolist()
    })

# CSV'ye yaz
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "weigh_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("✅ Dönüştürülmüş dosya kaydedildi:", transformed_file_path)


# 107. extended version of weigh transform

In [ ]:
import pandas as pd
import numpy as np
import sympy as sp

# Weigh transform using sympy
def trm_weigh(A):
    m, n = A.shape
    x, y = sp.symbols('x y')
    B = 0

    # Construct the formal power series expression
    for i in range(m):
        for j in range(n):
            k = 1
            z = 1
            ki, kj = i, j
            while ki < m and kj < n:
                B += A[i, j] * z / k * (x ** ki) * (y ** kj)
                k += 1
                z = -z
                ki += 1
                kj += 1

    # Exponentiate the series
    B_exp = sp.exp(B)

    # Extract coefficients for each (i, j)
    result = np.zeros((m, n), dtype=object)
    B_exp_series = B_exp.series(x, 0, m).removeO().series(y, 0, n).removeO()
    for i in range(m):
        for j in range(n):
            coeff = B_exp_series.coeff(x, i).coeff(y, j)
            result[i, j] = coeff

    return result

# Extended version with exponential factor
def trm_weigh_x(A):
    a = 2 ** A[0, 0]
    return a * trm_weigh(A)

# Read sequences from CSV
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply the extended weigh transform
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    matrix_size = int(len(original) ** 0.5)
    if matrix_size == 0:
        continue
    matrix = np.array(original[:matrix_size**2]).reshape((matrix_size, matrix_size))
    transformed_matrix = trm_weigh_x(matrix)
    # Convert sympy elements to strings to save to CSV
    transformed_flat = [str(val) for val in transformed_matrix.flatten()]
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "ExtendedWeighTransformedMatrix": transformed_flat
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_df.to_csv("extended_weigh_transformed.csv", index=False)


# 108. inverse weigh transform for matrices

In [ ]:
import pandas as pd
import numpy as np
import sympy as sp

# Inverse Weigh Transform for matrices
def trm_i_weigh(A):
    m, n = A.shape
    x, y = sp.symbols('x y')

    # Step 1: Reconstruct the generating function
    B = 0
    for i in range(m):
        for j in range(n):
            B += A[i, j] * (x ** i) * (y ** j)

    # Step 2: Logarithm of (1 + B)
    B_log = sp.log(1 + B)

    # Step 3: Extract coefficients back into a matrix
    result = np.zeros((m, n), dtype=object)
    B_series = B_log.series(x, 0, m).removeO().series(y, 0, n).removeO()
    for i in range(m):
        for j in range(n):
            coeff = B_series.coeff(x, i).coeff(y, j)
            result[i, j] = coeff

    # Step 4: Alternating subtraction as specified (optional refinement)
    for i in range((m - 1) // 2 + 1):
        for j in range((n - 1) // 2 + 1):
            k = 2
            z = -1
            ki = i + i
            kj = j + j
            while ki < m and kj < n:
                result[ki, kj] -= result[i, j] * z / k
                k += 1
                z = -z
                ki += i
                kj += j

    return result

# CSV okuma
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Transform uygulama
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    matrix_size = int(len(original) ** 0.5)
    if matrix_size == 0:
        continue
    matrix = np.array(original[:matrix_size**2]).reshape((matrix_size, matrix_size))
    transformed_matrix = trm_i_weigh(matrix)
    
    # CSV için string'e çevir
    flat = [str(x) for x in transformed_matrix.flatten()]
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseWeighTransformedMatrix": flat
    })

# CSV'ye yaz
transformed_df = pd.DataFrame(transformed_sequences)
output_path = "inverse_weigh_transformed.csv"
transformed_df.to_csv(output_path, index=False)

print("CSV kaydedildi:", output_path)


# 109. extended version of inverse weigh transform

In [ ]:
import pandas as pd
import numpy as np
import sympy as sp

def trm_i_weigh(A):
    m, n = A.shape
    x, y = sp.symbols('x y')
    
    # Step 1: polynomial summation
    B = 0
    for i in range(m):
        for j in range(n):
            B += A[i, j] * (x**i) * (y**j)
    
    # Step 2: logarithm
    B = sp.log(1 + B)
    
    # Step 3: extract coefficients
    result = np.zeros((m, n), dtype=object)
    B_series = B.series(x, 0, m).removeO().series(y, 0, n).removeO()
    for i in range(m):
        for j in range(n):
            coeff = B_series.coeff(x, i).coeff(y, j)
            result[i, j] = coeff
    
    # Step 4: alternating subtraction
    for i in range((m - 1) // 2 + 1):
        for j in range((n - 1) // 2 + 1):
            k = 2
            z = -1
            ki = i + i
            kj = j + j
            while ki < m and kj < n:
                result[ki, kj] -= result[i, j] * z / k
                k += 1
                z = -z
                ki += i
                kj += j
    
    return result

def trv_i_weigh_x(A):
    a = A[0, 0]
    if a == 0:
        raise ValueError("First element 'a' is zero, log undefined")
    
    b = sp.log(a) / sp.log(2)
    
    # Normalize A by a, convert to sympy Matrix for symbolic calculation
    A_sym = sp.Matrix(A / a)
    
    # Apply inverse weigh transform
    result = trm_i_weigh(A_sym)
    
    # Set first element
    result[0, 0] = b
    
    return result

# CSV okuma ve işlem
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    matrix_size = int(len(original) ** 0.5)
    if matrix_size == 0:
        continue
    matrix = np.array(original[:matrix_size**2]).reshape((matrix_size, matrix_size))
    try:
        transformed_matrix = trv_i_weigh_x(matrix)
    except Exception as e:
        print(f"Error processing sequence {entry['SequenceID']}: {e}")
        continue
    
    flat = [str(x) for x in np.array(transformed_matrix).flatten()]
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "ExtendedInverseWeighTransformedMatrix": flat
    })

transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "extended_inverse_weigh_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("CSV kaydedildi:", transformed_file_path)


# 110. weigh product transform

In [7]:
import pandas as pd
import numpy as np

# Weigh Product Transform function
def trv_weigh_p(A):
    n = len(A)
    
    # Step 1: Initialize the vector B
    B = np.zeros(n-1)  # Vector B is of length n-1 (excluding A[0] and A[1])
    
    # Step 2: Perform the main loop and transformation
    A = A[1:]  # Remove the first two elements (A[0] and A[1])
    for i in range(2, n):
        s = np.math.factorial(i) * A[i-1]  # Equivalent to bigomega(i) * A[i]
        j = i
        while j <= n:
            B[j-1] += s
            j *= i
            s = -s  # Flip the sign
     
    # Step 3: Update the array A
    A_new = np.zeros(n)  # New A with length n (including A[0] and A[1])
    A_new[0] = 0
    A_new[1] = 1
    
    for i in range(2, n):
        A_new[i] = np.sum([A[j-1] * B[i//(j+1)] for j in range(1, n) if i % (j+1) == 0])
    
    return A_new

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply weigh product transform to each sequence
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    
    # Apply the weigh product transform
    transformed = trv_weigh_p(original)
    
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "WeighProductTransformed": transformed.tolist()  # Store as list for easy CSV saving
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "weigh_product_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_31480\1577363131.py:14: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  s = np.math.factorial(i) * A[i-1]  # Equivalent to bigomega(i) * A[i]


IndexError: index 99 is out of bounds for axis 0 with size 99

# 111. inverse weigh product transform

In [8]:
import pandas as pd
import numpy as np

# Inverse Weigh Product Transform function
def trv_i_weigh_p(A):
    n = len(A)
    
    # Step 1: Initialize the vector B
    B = np.zeros(n-1)  # Vector B is of length n-1 (excluding A[0] and A[1])
    
    # Step 2: Apply the transformation
    A = A[1:]  # Remove the first two elements (A[0] and A[1])
    for i in range(2, n):
        B[i-1] = np.math.factorial(i) * A[i-1] - np.sum([A[j-1] * B[i//(j+1)] for j in range(1, n) if i % (j+1) == 0])

    # Step 3: Adjust using sqrtint and updating B
    for i in range(2, int(np.sqrt(n)) + 1):
        j = i**2
        s = B[i-1]
        while j <= n:
            B[j-1] += s
            s = -s
            j *= i

    # Step 4: Update A
    A_new = np.zeros(n)  # New A with length n (including A[0] and A[1])
    A_new[0] = 0  # b_0 = 0
    for i in range(2, n):
        A_new[i] = B[i-1] / np.math.factorial(i)

    return A_new

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply inverse weigh product transform to each sequence
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    
    # Apply the inverse weigh product transform
    transformed = trv_i_weigh_p(original)
    
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseWeighProductTransformed": transformed.tolist()  # Store as list for easy CSV saving
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "inverse_weigh_product_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_31480\193741867.py:14: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  B[i-1] = np.math.factorial(i) * A[i-1] - np.sum([A[j-1] * B[i//(j+1)] for j in range(1, n) if i % (j+1) == 0])
C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_31480\193741867.py:29: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  A_new[i] = B[i-1] / np.math.factorial(i)


IndexError: index 99 is out of bounds for axis 0 with size 99

# 112. weigh_2 transform

In [9]:
import pandas as pd
import numpy as np

# Weigh 2 Transform function
def trv_weigh_2(A):
    n = len(A)
    
    # Step 1: Create the generating set B using Ser(A,x)
    x = np.array(A)  # Create the sequence as numpy array
    B = np.polynomial.Polynomial(A)  # B is the polynomial representation of A
    
    # Step 2: Apply the transformation (B^2 - subst(B, x, x^2)) / 2
    B = (B ** 2 - B(x) ** 2) / 2
    
    # Step 3: Extract the coefficients of B and update A
    A_new = np.zeros(n)  # Initialize a new array A with zeros
    for i in range(n):
        A_new[i] = B.coef[i]
    
    return A_new

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply weigh_2 transform to each sequence
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    
    # Apply the weigh_2 transform
    transformed = trv_weigh_2(original)
    
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "Weigh2Transformed": transformed.tolist()  # Store as list for easy CSV saving
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "weigh_2_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_31480\3279429318.py:13: RuntimeWarning: overflow encountered in square
  B = (B ** 2 - B(x) ** 2) / 2


IndexError: index 1 is out of bounds for axis 0 with size 1

# 113. weigh_2 transform for matrices

In [10]:
import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import Polynomial

# Weigh 2 Transform for matrices function
def trm_weigh_2(A):
    m, n = len(A), len(A[0])  # Get dimensions of the matrix
    
    # Initialize polynomials for B and B2
    B = np.zeros((m, n))  # B will store the first sum
    B2 = np.zeros((m, n))  # B2 will store the second sum

    # Step 1: Compute the first sum (B)
    x, y = np.array([1]), np.array([1])  # Assuming x and y as 1 for simplicity
    for i in range(m):
        for j in range(n):
            B[i, j] += A[i][j] * (x ** i) * (y ** j)

    # Step 2: Compute the second sum (B2)
    for i in range(m // 2):
        for j in range(n // 2):
            B2[i, j] += A[i][j] * (x ** (2 * i)) * (y ** (2 * j))

    # Step 3: Combine the sums B and B2 as per the formula (B^2 - B2) / 2
    B = (B ** 2 - B2) / 2

    # Step 4: Update A with the polynomial coefficients
    for i in range(m):
        for j in range(n):
            A[i][j] = Polynomial(B[i, j]).coef[i]

    return A

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply weigh_2 transform to each sequence (assuming they represent matrices)
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    
    # Assume each sequence is a square matrix of size (n x n)
    size = int(len(original) ** 0.5)  # Assume square matrices for simplicity
    matrix = np.array(original[:size*size]).reshape((size, size))  # Reshape the sequence into a matrix
    
    # Apply the weigh_2 transform to the matrix
    transformed_matrix = trm_weigh_2(matrix)
    
    # Flatten the transformed matrix for storage
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "Weigh2TransformedMatrix": transformed_matrix.flatten().tolist()  # Flatten and store as a list
    })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "weigh_2_transformed_matrices.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_31480\2267010702.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  B[i, j] += A[i][j] * (x ** i) * (y ** j)
C:\Users\ÇELİK\AppData\Local\Temp\ipykernel_31480\2267010702.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  B2[i, j] += A[i][j] * (x ** (2 * i)) * (y ** (2 * j))


IndexError: index 1 is out of bounds for axis 0 with size 1

# 114. weigh diffsize transform

Analiz kısa sürüyor, fakat sonuçlar hep yalnış çıkıyor.

# 115. inverse weigh diffsize transform

Analiz kısa sürüyor, fakat sonuçlar hep yalnış çıkıyor.

# 116. witt transform

Uzun dizilerde sorun çıkıyor, dönüşümü yapamıyor. Bunun için dizilerin ilk 100 elemanına dönüşüm uygulandı. Tüm sonuçlar doğru bulundu.

In [18]:
import pandas as pd
import numpy as np

def trv_witt(A):
    """
    GP/PARI'deki trv_witt fonksiyonunun doğru Python çevirisi.
    A dizisinin ilk elemanı (a₀) yok sayılır ve dönüşte başına 0 eklenir.
    """
    A = A[1:100]  # a₀'ı yok say, ilk 100 elemanı al
    n = len(A)
    A = A.copy()
    C = np.zeros(n, dtype=object)  # Büyük sayı desteği için dtype=object

    for i in range(1, (n // 2) + 1):
        for j in range(i - 1, n, i):
            C[j] = 0

        v = A[i - 1]
        k = 1
        for j in range(i - 1, n, i):
            C[j] += v
            if k > 1:
                A[j] -= C[j] // k
            for m in range(j + j + 1, n, j + 1):
                C[m] -= C[j]
            k += 1
            v *= A[i - 1]

    return [0] + A  # Başına 0 ekle


def read_sequences_from_csv(filepath):
    sequences = []
    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if line.startswith("#") or not line.strip():
                continue
            parts = line.strip().split(",")
            seq_id = parts[0].strip()
            try:
                values = [int(x) for x in parts[1:] if x.strip()]
                values = values[:100]  # İlk 100 elemanı al
                sequences.append({"SequenceID": seq_id, "Sequence": values})
            except ValueError:
                continue
    return sequences


def transform_sequences(sequences):
    transformed = []
    for entry in sequences:
        seq = entry["Sequence"]
        if len(seq) < 3:
            continue
        try:
            new_seq = trv_witt(seq)
            transformed.append({
                "SequenceID": entry["SequenceID"],
                "WittTransformed": new_seq
            })
        except Exception as e:
            print(f"Hata: {entry['SequenceID']} - {str(e)}")
    return transformed


if __name__ == "__main__":
    sequences = read_sequences_from_csv("stripped.csv")
    transformed_sequences = transform_sequences(sequences)

    df = pd.DataFrame(transformed_sequences)
    df.to_csv("witt_transformed_limited2.csv", index=False)


# 117. inverse witt transform

Tüm dizilere tam olarak uygulandı ve doğru sonuç bulunudu.

In [22]:
import pandas as pd
import numpy as np

def trv_i_witt(A):
    """
    GP/Pari'deki trv_i_witt fonksiyonunun birebir karşılığı.
    A dizisi 0-indeksli, A[0] = a_0 olmalı (ama a_0 göz ardı edilecek).
    Fonksiyon daima b_0 = 0 olacak şekilde başlar.
    """
    n = len(A)
    if n < 2:
        return [0]

    # GP'de a_0 (yani A[0]) yok sayılıyor; A[1] = a_1
    A = A[1:]  # a_1, a_2, ...

    m = len(A)
    B = [0] * m
    C = [0] * m

    for i in range(1, m + 1):  # GP'de 1-based index
        for j in range(i - 1, m, i):
            C[j] = 0
        v = A[i - 1]
        k = 1
        for j in range(i - 1, m, i):
            C[j] += v
            B[j] += C[j] // k  # Integer division
            for m_idx in range(j + j + 1, m, j + 1):
                C[m_idx] -= C[j]
            k += 1
            v *= A[i - 1]

    return [0] + B


# CSV okuma ve dönüştürme işlemleri
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Dönüştürme
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) < 3:
        continue
    transformed = trv_i_witt(original)
    transformed_sequences.append({
        "SequenceID": entry["SequenceID"],
        "InverseWittTransformed": transformed
    })

# CSV'ye kaydet
transformed_df = pd.DataFrame(transformed_sequences)
transformed_df.to_csv("inverse_witt_transformed.csv", index=False)


# 118. and convolution operation

Sonuçlar tam doğru çıktı, hepsi doğru.

In [23]:
import pandas as pd
import numpy as np

# Convolution AND operation function
def opv_conv_and(A, B):
    C_size = min(len(A), len(B))
    C = np.zeros(C_size, dtype=object)  # dtype=object to handle big integers

    for i in range(C_size - 1, -1, -1):
        total = 0
        for j in range(i + 1):
            total += int(A[j]) & int(B[i - j])
        C[i] = total

    return C

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply convolution AND operation and store the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]

    if len(original) >= 2:
        transformed = opv_conv_and(original, original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "ConvolutionANDResult": transformed.tolist()
        })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "convolution_and_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


'convolution_and_transformed.csv'

# 119. gcd convolution operation
Sonuçlar tam doğru çıktı, hepsi doğru.

In [24]:
import pandas as pd
from math import gcd

# GCD Convolution operation (pure Python, avoids OverflowError)
def opv_conv_gcd(A, B):
    C_size = min(len(A), len(B))
    C = []

    for i in range(C_size):
        total = 0
        for j in range(i + 1):
            total += gcd(A[j], B[i - j])
        C.append(total)
    return C

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply GCD convolution and store results
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) >= 2:
        transformed = opv_conv_gcd(original, original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "GCDConvolutionResult": transformed
        })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "gcd_convolution_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print(f"✔️ GCD convolution işlemi tamamlandı. Dosya: {transformed_file_path}")


✔️ GCD convolution işlemi tamamlandı. Dosya: gcd_convolution_transformed.csv


# 120. lcm convolution operation
Sonuçlar tam doğru çıktı, hepsi doğru.

In [25]:
import pandas as pd
import numpy as np
from math import gcd

# Güvenli LCM (Least Common Multiple) function
def lcm(a, b):
    if a == 0 or b == 0:
        return 0
    return abs(a * b) // gcd(a, b)

# LCM Convolution operation function
def opv_conv_lcm(A, B):
    C_size = min(len(A), len(B))
    C = np.zeros(C_size, dtype=object)  # object olarak ayarlıyoruz çünkü LCM hızlıca büyüyebilir

    for i in range(C_size - 1, -1, -1):
        total = 0
        for j in range(i + 1):
            total += lcm(int(A[j]), int(B[i - j]))
        C[i] = total

    return C

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply LCM convolution operation and store the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]

    if len(original) >= 2:
        transformed = opv_conv_lcm(original, original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "LCMConvolutionResult": transformed.tolist()
        })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "lcm_convolution_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

transformed_file_path


'lcm_convolution_transformed.csv'

# 121. or convolution operation

Sonuçlar tam doğru çıktı, hepsi doğru.

In [26]:
import pandas as pd
import numpy as np

# Bitwise OR Convolution operation function
def opv_conv_or(A, B):
    C_size = min(len(A), len(B))
    C = np.zeros(C_size, dtype=object)  # dtype=object ile büyük int'leri destekle
    
    for i in range(C_size - 1, -1, -1):
        C[i] = sum(A[j] | B[i - j] for j in range(i + 1))
    
    return C

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply OR convolution operation and store the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    
    if len(original) >= 2:
        transformed = opv_conv_or(original, original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "ORConvolutionResult": transformed.tolist()
        })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "or_convolution_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("Dönüşüm tamamlandı:", transformed_file_path)


Dönüşüm tamamlandı: or_convolution_transformed.csv


# 122. xor convolution operation
Sonuçlar tam doğru çıktı, hepsi doğru.

In [27]:
import pandas as pd
import numpy as np

# XOR Convolution operation function
def opv_conv_xor(A, B):
    C_size = min(len(A), len(B))
    C = np.zeros(C_size, dtype=object)  # object tipi kullan
    
    for i in range(C_size - 1, -1, -1):
        C[i] = sum(A[j] ^ B[i - j] for j in range(i + 1))
    
    return C

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply XOR convolution operation and store the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    
    if len(original) >= 2:
        transformed = opv_conv_xor(original, original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "XORConvolutionResult": transformed.tolist()
        })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "xor_convolution_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("XOR dönüşümü tamamlandı:", transformed_file_path)


XOR dönüşümü tamamlandı: xor_convolution_transformed.csv


# 123.xor division operation

Sonuçlar tam doğru çıktı, hepsi doğru.

In [28]:
import pandas as pd
import numpy as np

# XOR Division operation function
def opv_div_xor(A, B):
    C_size = min(len(A), len(B))
    C = np.zeros(C_size, dtype=object)  # object tipi

    for i in range(C_size):
        total = 0
        for j in range(i):
            total += A[i - j] ^ C[j]
        C[i] = (A[0] ^ B[i]) - total
    
    return C

# Read stripped.csv and parse sequences
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Apply XOR division operation and store the transformed sequences
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    
    if len(original) >= 2:
        transformed = opv_div_xor(original, original)
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "XORDivisionResult": transformed.tolist()
        })

# Save to CSV
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "xor_division_transformed.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("XOR division dönüşümü tamamlandı:", transformed_file_path)


XOR division dönüşümü tamamlandı: xor_division_transformed.csv


# 124. cycle "boy-girl" operation

Analiz hızlı oluyor, fakat sonuçlar GPARI ile uyuşmuyor.

# 125. euler "type" operation
Analiz hızlı oluyor, fakat sonuçlar GPARI ile uyuşmuyor.

# 126. exponential "type" operation
Sonuçlar tam doğru çıktı, hepsi doğru.

In [36]:
import pandas as pd
from fractions import Fraction

def opv_mul_ed(A, B):
    return [a * b for a, b in zip(A, B)]

def trv_log(A):
    B = [Fraction(0)] * len(A)
    for n in range(1, len(A)):
        B[n] = Fraction(A[n], n + 1)
    return B

def trv_exp(A):
    B = [Fraction(0)] * len(A)
    B[0] = Fraction(1)
    for n in range(1, len(A)):
        B[n] = A[n - 1] * n
    return B

def opv_exp_type(A, B):
    len_min = min(len(A), len(B))
    A = list(map(Fraction, A[:len_min]))
    B = list(map(Fraction, B[:len_min]))
    return trv_exp(opv_mul_ed(trv_log(A), B))

# CSV'den veri oku
sequences = []
with open("stripped.csv", "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split(",")
        seq_id = parts[0].strip()
        try:
            values = [int(x) for x in parts[1:] if x.strip()]
            sequences.append({"SequenceID": seq_id, "Sequence": values})
        except ValueError:
            continue

# Transformasyon uygula
transformed_sequences = []
for entry in sequences:
    original = entry["Sequence"]
    if len(original) >= 2:
        transformed = opv_exp_type(original, original)
        # İsteğe bağlı olarak float yerine tam sayı çıktı üretilebilir:
        transformed_str = [int(x) if x.denominator == 1 else float(x) for x in transformed]
        transformed_sequences.append({
            "SequenceID": entry["SequenceID"],
            "ExponentialTypeResult": transformed_str
        })

# Sonucu CSV'ye yaz
transformed_df = pd.DataFrame(transformed_sequences)
transformed_file_path = "exponential_type_transformed_formal.csv"
transformed_df.to_csv(transformed_file_path, index=False)

print("Dönüştürülmüş dosya:", transformed_file_path)


Dönüştürülmüş dosya: exponential_type_transformed_formal.csv


# 127. invert "boy-girl" operation

Aşağıdaki kod yalnış sonuç veriyor. Daha sonra tekrar bak. Dönüşüm sonucnu tam olarak bielemdeim. GPARI ile karşılaştırmada emin olamadım.